In [ ]:
from brian2 import *
import csv
import random
import os, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
from scipy.spatial import distance

In [ ]:
# Interesting links:

# http://www.maths.dit.ie/~johnbutler/Izhikevich/IzhikevichModel.html#Thalamo-cortical
# https://www.izhikevich.org/publications/spikes.pdf
# https://github.com/brian-team/brian2/issues/809
# http://neuralensemble.org/docs/PyNN/reference/neuronmodels.html

In [ ]:
min_range_V = 5
max_range_V = 10

In [ ]:
list_attack_generation = ["FLO","SCA"]

### Methods for csv export

In [ ]:
def append_to_csv_file(filename, line):
    # Update the attack file
    with open(filename, 'a') as csvFile:
        writer = csv.writer(csvFile, delimiter=';')
        writer.writerow(line)

def dump_simulation_data_to_csv(attack, test, n_attacks, n_neurons, attacked_neurons, coord_attack, stim_value, n_exec, vIncrement, paramI, monitor, filename):
    mon_trains = monitor.spike_trains()
    
    for neuron in mon_trains.keys():
        for time_delta in mon_trains[neuron]:
            append_to_csv_file(filename, [attack, test, n_attacks, n_neurons, attacked_neurons, coord_attack, stim_value, n_exec, str(vIncrement), str(paramI), round(time_delta/ms, 1), neuron])

def list_neurons_to_string(list_neurons):
    result = ""
    for neuron in list_neurons:
        result += str(neuron)+"-"
        
    return result[:-1]

### General variables and constants

In [ ]:
list_coordinates_optimal_path = [[0,0], [1,0], [1,1], [1,2], [0,2], [0,3], [0,4], [0,5], [1,5], [2,5], 
                                 [2,4], [2,3], [3,3], [3,2], [3,1], [3,0], [4,0], [5,0], [6,0], [6,1], 
                                 [6,2], [5,2], [5,3], [5,4], [5,5], [6,5], [6,6]]

In [ ]:
#STEP_TIME = 25*ms
STEP_TIME = 1000*ms

# STEP_TIME per movement. As there are 27 positions -> 27*STEP_TIME. 
SIMULATION_TIME = (STEP_TIME/ms*len(list_coordinates_optimal_path))*ms

In [ ]:
SIMULATION_TIME

In [ ]:
# Duration of simulation, after STEP_TIME. Divided by the number of neurons
def get_time_steps_sequential(tAttack):
    return trunc(((SIMULATION_TIME-tAttack)/200)/ms)*ms

def get_number_attacks_per_position_sequential(tAttack):
    return STEP_TIME/get_time_steps_sequential(tAttack)

def get_last_instant_attack_sequential(tAttack):
    return tAttack+get_time_steps_sequential(tAttack)*200

In [ ]:
get_time_steps_sequential(50*ms)

In [ ]:
get_number_attacks_per_position_sequential(50*ms)

In [ ]:
get_last_instant_attack_sequential(50*ms)

### Auxiliary methods to translate between coordinates and indexes

In [ ]:
def generate_list_random_neurons(nNeurons):
    list_neurons = list(range(0, 200))
    result = []

    for i in range(0, nNeurons):
        index = randint(0, len(list_neurons))
        
        result.append(list_neurons[index])
        del list_neurons[index]
    
    return sorted(result)

In [ ]:
# Dict that stores the index value of a neuron: 3D coordinate -> numeric index (0-199)
dict_neurons_to_numbers = {}

counter = 0

for i in range(0, 5):
    for j in range(0, 5): 
        for k in range(0, 8): 
            dict_neurons_to_numbers[(i,j,k)] = counter
            counter+=1

In [ ]:
#dict_neurons_to_numbers

In [ ]:
# Dict that stores the instant in which the mouse is in eah position of the optimal path

dict_instant_optimal_path = {}

counter = 0

for coord in list_coordinates_optimal_path:
    dict_instant_optimal_path[(coord[0],coord[1])]= counter
    counter += STEP_TIME/ms     

# List of the instants previously calculated (list values of dict)
list_values = list(dict_instant_optimal_path.values())
list_instant_optimal_path = []

for value in list_values:
    list_instant_optimal_path.append(round(value))

In [ ]:
def get_visible_coordinates_by_position(x,y):
    
    coords = [[x-1,y-1], [x-1, y], [x-1,y+1], [x,y-1], [x,y], [x,y+1], [x+1,y-1], [x+1,y], [x+1,y+1]]
    result = []
    
    for n in coords:
        if (0 <= n[0] <= 6) and (0 <= n[1] <= 6):
            result.append(n)
            
    return result

In [ ]:
#get_visible_coordinates_by_position(3,3)

In [ ]:
# Get the neurons indexes that are related to a given maze coordinate
def get_neurons_indexes_by_position(x, y):
    
    coords = [
        [x-2,y-2,0], [x-2,y-2,1], [x-2,y-2,2], [x-2,y-2,3], [x-2,y-2,4], [x-2,y-2,5], [x-2,y-2,6], [x-2,y-2,7],
        [x-2,y-1,0], [x-2,y-1,1], [x-2,y-1,2], [x-2,y-1,3], [x-2,y-1,4], [x-2,y-1,5], [x-2,y-1,6], [x-2,y-1,7],
        [x-2,y,0], [x-2,y,1], [x-2,y,2], [x-2,y,3], [x-2,y,4], [x-2,y,5], [x-2,y,6], [x-2,y,7],
        [x-1,y-2,0], [x-1,y-2,1], [x-1,y-2,2], [x-1,y-2,3], [x-1,y-2,4], [x-1,y-2,5], [x-1,y-2,6], [x-1,y-2,7],
        [x-1,y-1,0], [x-1,y-1,1], [x-1,y-1,2], [x-1,y-1,3], [x-1,y-1,4], [x-1,y-1,5], [x-1,y-1,6], [x-1,y-1,7],
        [x-1,y,0], [x-1,y,1], [x-1,y,2], [x-1,y,3], [x-1,y,4], [x-1,y,5], [x-1,y,6], [x-1,y,7],
        [x,y-2,0], [x,y-2,1], [x,y-2,2], [x,y-2,3], [x,y-2,4], [x,y-2,5], [x,y-2,6], [x,y-2,7],
        [x,y-1,0], [x,y-1,1], [x,y-1,2], [x,y-1,3], [x,y-1,4], [x,y-1,5], [x,y-1,6], [x,y-1,7],
        [x,y,0], [x,y,1], [x,y,2], [x,y,3], [x,y,4], [x,y,5], [x,y,6], [x,y,7],
    ]
    
    coord_result = []
    result = []
    
    for n in coords:
        if (0 <= n[0] <= 4) and (0 <= n[1] <= 4):
            coord_result.append(n)
            
    for coord in coord_result:
        result.append(dict_neurons_to_numbers[(coord[0],coord[1],coord[2])])
            
    return result

In [ ]:
#get_neurons_indexes_by_position(0,0)

In [ ]:
def get_related_neurons_visible_positions(x, y):
    # Get the positions visible from the current position
    visible_coords = get_visible_coordinates_by_position(x, y)
    
    # Get the complete list of neurons related to all visible positions
    list_neurons_index = []

    for coord in visible_coords:
        neurons = get_neurons_indexes_by_position(coord[0], coord[1])
        
        for n in neurons:
            list_neurons_index.append(n)
    
    # Remove duplicates in list_neurons_index
    result = []
    
    for n in list_neurons_index:
        if n not in result:
            result.append(n)
            
    return result

In [ ]:
#get_related_neurons_visible_positions(0,2)

In [ ]:
# Get all neurons related to all positions, in a dict. Both list of neurons and count
dict_all_neurons = {}
dict_all_neurons_count = {}

counter = 0

for pos in list_coordinates_optimal_path:
    dict_all_neurons[counter] = get_related_neurons_visible_positions(pos[0], pos[1])
    dict_all_neurons_count[counter] = len(get_related_neurons_visible_positions(pos[0], pos[1]))
    
    counter+=1

### Auxiliary methods for Brian Simulation

In [ ]:
# Print plot for a state monitor
def plot_state_monitor(monitor, neuron):
    plot(monitor.t/ms, monitor.v[neuron]/mV)
    xlabel('Time (ms)')
    ylabel('v');
    
sns.set(style="whitegrid",font_scale=2.5, rc={'figure.figsize':(60,20)})


def seaborn_state_monitor(monitor, neuron): 
    ax = sns.lineplot(monitor.t/ms, monitor.v[neuron]/mV, linewidth=3.0)
    
    '''
    cont = STEP_TIME/ms
    for a in range(0, len(list_coordinates_optimal_path)+1):
        plt.axvline(x=cont*a, color="red")

    ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
    '''
    plt.plot()

In [ ]:
### FUNCTIONS TO LOAD DATA FROM EXTERNAL FILES (TOPOLOGY, WEIGHTS, PARAMETERS...) ###

def getSynapsisDataFromFile(filename):
    synapsysData = []
    minWeight = 0
    maxWeight = 0
    
    with open(filename) as csvfile:
        csvReader = csv.reader(csvfile, delimiter=';')
        next(csvReader, None) # skip header
        
        firstRow =  next(csvReader)
        synapsysData.append([int(firstRow[0]), int(firstRow[1]), float(firstRow[2])])
        minWeight = float(firstRow[2])
        maxWeight = float(firstRow[2])
        
        for row in csvReader:
            synapsysData.append([int(row[0]), int(row[1]), float(row[2])])
            
            if minWeight > float(row[2]):
                minWeight = float(row[2])
            if maxWeight < float(row[2]):
                maxWeight = float(row[2])
    
    return synapsysData, minWeight, maxWeight

def loadIzhikevichParamI(fileName):
    with open(fileName, 'r') as reader:
        I = [line.rstrip('\n') for line in reader]
        
        for i in range(0, len(I)):
            I[i] = float(I[i])*mV/ms
        
        return I
    
def load_initial_voltages(fileName):
    with open(fileName, 'r') as reader:
        v = [line.rstrip('\n') for line in reader]
        
        for i in range(0, len(v)):
            v[i] = float(v[i])*mV
        
        return v
    
#I = loadIzhikevichParamI("paramI.txt")


def exportDataStateMonitor(dataStateMon):
    dataStoreStateMon = []
    
    # STATEMON
    # attr (v), time (0-999), neuron (0-199)
    # if execTime == 100ms, len is 1k iterations
    for time in range(0, len(dataStateMon['v'])):
        timeData = []

        for neuron in range(0, len(dataStateMon['v'][time])):
            timeData.append(dataStateMon['v'][time][neuron])
        
        dataStoreStateMon.append(timeData)

     # Result: for each instant, the V of each single neuron
    
    return dataStoreStateMon


def exportDataSpikeMonitor(dataSpikeMon):          
    # SPIKEMON
    # "t": time of each spike during the simulation
    # "i": nb of neuron that spikes, in chronological order. 
    # "count": indicates, for each neuron, the nb of spikes during the simulation

    # Result: array of 2 positions that contains:
    #     0) array where each position has the binome <time instant, nb of neuron>
    #     1) the array "count" with the nb of spikes per neuron
    #
    # Precondition: nb elements in "t" == nb elments in "i"
    
    dataStoreSpikeMon = []
    timeSpikeMon = []
    countSpikeMon = []

    for time in range(0, len(dataSpikeMon['t'])):
        # Each position: <time, nNeuron>
        timeSpikeMon.append([dataSpikeMon['t'][time], dataSpikeMon['i'][time]])
    
    for nSpikes in range(0, len(dataSpikeMon['count'])):
        # Each position: nSpikes
        countSpikeMon.append(dataSpikeMon['count'][nSpikes])
    
    dataStoreSpikeMon = [timeSpikeMon, countSpikeMon]
    
    return dataStoreSpikeMon


def exportAllDataMonitors(stateMons, spikeMons):
    # Generate data to export to CSV (suitable format for later processing)
    dataStateMon1 = exportDataStateMonitor(stateMons[0].get_states(units=False))
    dataStateMon2 = exportDataStateMonitor(stateMons[1].get_states(units=False))
    dataStateMon3 = exportDataStateMonitor(stateMons[2].get_states(units=False))

    dataSpikeMon1 = exportDataSpikeMonitor(spikeMons[0].get_states(units=False))
    dataSpikeMon2 = exportDataSpikeMonitor(spikeMons[1].get_states(units=False))
    dataSpikeMon3 = exportDataSpikeMonitor(spikeMons[2].get_states(units=False))
    
    return [dataStateMon1, dataSpikeMon1, dataStateMon2, dataSpikeMon2, dataStateMon3, dataSpikeMon3]

### FUNCTIONS TO GENERATE RANDOM NUMBER OF NEURONS ###

def generateRandomNeurons(currentNeurons):
    selectedNeurons = []
    
    # Generate random coordinates for the current number of neurons
    for currentRandom in range(0, currentNeurons):

        randNeuron = np.random.randint(0, 200)

        # Avoid duplicates
        while (randNeuron in selectedNeurons):
            randNeuron = np.random.randint(0, 200)

        # Store the selected neuron
        selectedNeurons.append(randNeuron)
        
    return selectedNeurons

    
def generatePairsRandomNeurons(currentNeurons):    
    selectedNeurons = []
    selectedPairsNeurons = []
    
    # Generate random coordinates for the current number of neurons -> pairs of neurons
    for currentRandom in range(0, currentNeurons):
        # Neuron 1 of the pair
        randNeuron1 = np.random.randint(0, 200)

        # Avoid duplicates for neuron 1
        while (randNeuron1 in selectedNeurons):
            randNeuron1 = np.random.randint(0, 200)

        selectedNeurons.append(randNeuron1)

        # Neuron 2 of the pair
        randNeuron2 = np.random.randint(0, 200)

        # Avoid duplicates for neuron 2
        while (randNeuron2 in selectedNeurons):
            randNeuron2 = np.random.randint(0, 200)

        selectedNeurons.append(randNeuron2)

        selectedPairsNeurons.append([randNeuron1, randNeuron2])
        
    return selectedPairsNeurons

### Load simulation parameters from files (weights, synapsis, I param...)

In [ ]:
### VARIABLES FOR THE SIMULATION OF THE ATTACKS ###

BASIC_MODEL = 0
IZHIKEVICH_MODEL = 1

# Load only once synapsis and weights
dataSynapsisConv1_Conv2, minWeightsConv1_Conv2, maxWeightsConv1_Conv2 = getSynapsisDataFromFile("synapsysConv1-Conv2.csv")
dataSynapsisConv2_Dense, minWeightsConv2_Dense, maxWeightsConv2_Dense = getSynapsisDataFromFile("synapsysConv2-Dense.csv")

# Process the data for the simulator
initSourceNeuronsConv1_Conv2 = []
initTargetNeuronsConv1_Conv2 = []
initSourceNeuronsConv2_Dense = []
initTargetNeuronsConv2_Dense = []
initWeightsMaze_Conv1 = []
initWeightsConv1_Conv2 = []
initWeightsConv2_Dense = []

for syn in range(0, len(dataSynapsisConv1_Conv2)):
    initSourceNeuronsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][0])
    initTargetNeuronsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][1])
    initWeightsConv1_Conv2.append(dataSynapsisConv1_Conv2[syn][2])

for syn in range(0, len(dataSynapsisConv2_Dense)):
    initSourceNeuronsConv2_Dense.append(dataSynapsisConv2_Dense[syn][0])
    initTargetNeuronsConv2_Dense.append(dataSynapsisConv2_Dense[syn][1])
    initWeightsConv2_Dense.append(dataSynapsisConv2_Dense[syn][2])


# Normalize weights for the Izhikevich model
np_initWeightsConv1_Conv2 = np.array(initWeightsConv1_Conv2)
np_initWeightsConv2_Dense = np.array(initWeightsConv2_Dense)

norm_initWeightsConv1_Conv2 = np.interp(np_initWeightsConv1_Conv2, (np_initWeightsConv1_Conv2.min(), np_initWeightsConv1_Conv2.max()), (min_range_V, max_range_V))
norm_initWeightsConv2_Dense = np.interp(np_initWeightsConv2_Dense, (np_initWeightsConv2_Dense.min(), np_initWeightsConv2_Dense.max()), (min_range_V, max_range_V))

N_NEURONS_MAX = 100
N_NEURONS_MIN = 1

attacks_dict = {
    'FLO': ['Flooding', N_NEURONS_MAX, [0.25, 0.5, 0.75, 1.0]], # Stim multiple neurons per t.u.
    'JAM': ['Jamming', N_NEURONS_MAX, [-1.0]], # Inhibit multiple neurons per t.u.
    'SCA': ['PortScanning', N_NEURONS_MIN, [0.25, 0.5, 0.75, 1.0]],  # Stim 1 neuron per t.u.
    'FOR': ['SelectiveForwarding', N_NEURONS_MIN, [-1.0]], # Inhibit 1 neuron per t.u.
    'SPO': ['Spoofing', int(N_NEURONS_MAX/2)], # the attack selects pairs of neurons -> double of this number
    'SYB': ['Sybil', N_NEURONS_MAX],
}

maze =  np.array([
   [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  0.,  1.,  0.], 
    [ 0.,  0.,  0.,  1.,  1.,  1.,  0.],
    [ 1.,  1.,  1.,  1.,  0.,  0.,  1.],
    [ 1.,  0.,  0.,  0.,  1.,  1.,  1.],
    [ 1.,  0.,  1.,  1.,  1.,  1.,  1.],
    [ 1.,  1.,  1.,  0.,  1.,  1.,  1.]
])


# Load initial voltages for all simulation (previously generated random values)
v_initial = load_initial_voltages("initial_voltage.txt")

### Normal simulation (without attacks)

In [ ]:
def spontaneous_simulation(simulationDuration, paramI):
    start_scope()
    
    #defaultclock.dt = 0.1*ms
    
    # Equations of the Izhikevich neuron model
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u + I : volt
    du/dt = a*(b*v-u) : volt/second
    I : volt/second
    a : Hz
    b : Hz
    c : volt
    d : volt/second
    neuronCounter : 1
    positionCounter : 1
    isFirstTime : 1
    timeCounter : second
    '''

    # Reset of Izhikevich model
    reset ='''
    v = c
    u += d
    '''

    thresholdValue = 30
    resetValue = -65

    # Definition of the 1st layer of the CNN
    G = NeuronGroup(276, eqs, threshold='v >= thresholdValue*mV', reset=reset, method='euler')
    # Initialise variables of all neurons (typical values)

    G.v = v_initial
    
    G.u = -13*mV/ms # b*v -> 0.2*-65 = -13
    G.a = 0.02/ms
    G.b = 0.2/ms
    G.c = resetValue*mV
    G.d = 8*mV/ms
    
    G.I = 10*mV/ms

    # Create subgroups
    layerConv1 = G[:200]
    layerConv2 = G[200:272]
    layerDense = G[272:276]

    layerConv1.neuronCounter[0] = 0
    layerConv1.positionCounter[0] = 0
    layerConv1.timeCounter[0] = 0*ms
    layerConv1.isFirstTime[0] = 1
        
    # Monitors G
    stateMonGlobal = StateMonitor(G, 'v', record=True)
    spikeMonGlobal = SpikeMonitor(G)
    
    # Synapsis definition
    synConv1_Conv2 = Synapses(layerConv1, layerConv2, 'w : volt', on_pre='v_post += w')
    synConv2_Dense = Synapses(layerConv2, layerDense, 'w : volt', on_pre='v_post += w')
    
    # Connect synapsis
    synConv1_Conv2.connect(i=initSourceNeuronsConv1_Conv2, j=initTargetNeuronsConv1_Conv2)
    synConv2_Dense.connect(i=initSourceNeuronsConv2_Dense, j=initTargetNeuronsConv2_Dense)
    
    synConv1_Conv2.w = norm_initWeightsConv1_Conv2*mV
    synConv2_Dense.w = norm_initWeightsConv2_Dense*mV
    
    
    @network_operation(dt=STEP_TIME)
    def periodicFunction():
        positionCounter = int(layerConv1.positionCounter[0])
        #print("current_counter: ", positionCounter)
        
        if layerConv1.isFirstTime[0] == 1:
            layerConv1.isFirstTime[0] = 0
        else:
            if positionCounter < len(list_coordinates_optimal_path):
                # Reset all neurons to default I value            
                #G.I = loadIzhikevichParamI("paramI.txt")   # Random values for all neurons
                G.I = 10*mV/ms
                
                # Get the coordinates of the current position
                coord = list_coordinates_optimal_path[positionCounter]

                # Update I value only for the neurons related to the visible positions from the current one
                list_neurons_index = get_related_neurons_visible_positions(coord[0], coord[1])

                for neuron in list_neurons_index:
                    G.I[neuron] = 15*mV/ms

                # Update the counter of the current position over the maze (for next iteration)
                layerConv1.positionCounter[0] += 1
    
    run(simulationDuration)
    
    return [spikeMonGlobal, stateMonGlobal]

### Simulation under attack

In [ ]:
# instantsAttack for FLO attack is an array of the timestamps where an attack occurs
# instantsAttack for SCA is just ONE timestamp, that indicates the starting instant of the attack
def attack_simulation(attack, isSequentialAttack, timeStepsSeqAttacks, instantsAttack, stimValue, simulationDuration, neuronList, vIncrement, paramI):
    start_scope()
    
    #defaultclock.dt = 0.1*ms
    
    # Interesting links:

    # https://www.izhikevich.org/publications/spikes.pdf
    # https://github.com/brian-team/brian2/issues/809
    # http://neuralensemble.org/docs/PyNN/reference/neuronmodels.html

    # Initial parameters for the equations (typical values)
    # a = 0.02/ms
    # b = 0.2/ms
    # c = -65*mV
    # d = 2*mV/ms
    # I = 0*mV/ms

    # Equations of the Izhikevich neuron model
    eqs = '''
    dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-u + I : volt
    du/dt = a*(b*v-u) : volt/second
    I : volt/second
    a : Hz
    b : Hz
    c : volt
    d : volt/second
    neuronCounter : 1
    positionCounter : 1
    isFirstTime : 1
    timeCounter : second
    '''

    # Reset of Izhikevich model
    reset ='''
    v = c
    u += d
    '''

    thresholdValue = 30
    resetValue = -65

    # Definition of the 1st layer of the CNN
    G = NeuronGroup(276, eqs, threshold='v >= thresholdValue*mV', reset=reset, method='euler')
    # Initialise variables of all neurons (typical values)

    G.v = v_initial
    
    G.u = -13*mV/ms # b*v -> 0.2*-65 = -13
    G.a = 0.02/ms
    G.b = 0.2/ms
    G.c = resetValue*mV
    G.d = 8*mV/ms
    #G.I = paramI*mV/ms
    #G.I = loadIzhikevichParamI("paramI.txt")
    G.I = 10*mV/ms
    
    # Create subgroups
    layerConv1 = G[:200]
    layerConv2 = G[200:272]
    layerDense = G[272:276]
    
    # We only store the counters for the first neuron of layer1 (more efficient)   
    layerConv1.neuronCounter[0] = 0
    layerConv1.positionCounter[0] = 0
    layerConv1.timeCounter[0] = 0*ms
    layerConv1.isFirstTime[0] = 1
    
    # Monitors G
    stateMonGlobal = StateMonitor(G, 'v', record=True)
    spikeMonGlobal = SpikeMonitor(G)
    
    # Synapsis definition
    synConv1_Conv2 = Synapses(layerConv1, layerConv2, 'w : volt', on_pre='v_post += w')
    synConv2_Dense = Synapses(layerConv2, layerDense, 'w : volt', on_pre='v_post += w')
    
    # Connect synapsis
    synConv1_Conv2.connect(i=initSourceNeuronsConv1_Conv2, j=initTargetNeuronsConv1_Conv2)
    synConv2_Dense.connect(i=initSourceNeuronsConv2_Dense, j=initTargetNeuronsConv2_Dense)
    
    synConv1_Conv2.w = norm_initWeightsConv1_Conv2*mV
    synConv2_Dense.w = norm_initWeightsConv2_Dense*mV   
    
    # Invoke method only when the rat changes the position
    @network_operation(dt=STEP_TIME)
    def periodicFunction():
        positionCounter = int(layerConv1.positionCounter[0])
        #print("current_counter: ", positionCounter)
        
        if layerConv1.isFirstTime[0] == 1:
            layerConv1.isFirstTime[0] = 0
        else:
            if positionCounter < len(list_coordinates_optimal_path):
                # Reset all neurons to default I value
                #G.I = loadIzhikevichParamI("paramI.txt")   # Random values for all neurons
                G.I = 10*mV/ms
                
                # Get the coordinates of the current position
                coord = list_coordinates_optimal_path[positionCounter]

                # Update I value only for the neurons related to the visible positions from the current one
                list_neurons_index = get_related_neurons_visible_positions(coord[0], coord[1])

                for neuron in list_neurons_index:
                    G.I[neuron] = 15*mV/ms

                # Update the counter of the current position over the maze (for next iteration)
                layerConv1.positionCounter[0] += 1
    
    @network_operation(dt=1*ms)
    def periodicFunctionMultipleNeurons():
        if(not isSequentialAttack):
            currentTimeCounter = int(layerConv1.timeCounter[0]*1000)
            
            # SPECIFIC TIME ON MULTIPLE RANDOM NEURONS
            currentTimesAttacks = []
            
            for instant in instantsAttack:
                currentTimesAttacks.append(int(instant/ms))
            
            #print("------")
            #print("currentTimeCounter: ", currentTimeCounter)
            #print("currentTimesAttacks: ", currentTimesAttacks)
            
            # Instant of the attack and current instant is >= STEP_TIME (fist position maze)
            if(currentTimeCounter in currentTimesAttacks): #& (layerConv1.timeCounter[0] >= STEP_TIME):
                #print("IF con time: ", currentTimeCounter)
                if (attack in ['FLO', 'JAM']):
                    #print("FLO/JAM")

                    for neuron in neuronList:
                        #print("neuron: ", neuron)
                        #print("G.v[neuron]: ", G.v[neuron])

                        if(attack=='FLO'):
                            layerConv1.v[neuron] += vIncrement*mV
                        else:
                            layerConv1.v[neuron] -= vIncrement*mV

                        #print("NEW G.v[neuron]: ", G.v[neuron])
                        #print("-------------------")
                    
                elif (attack == 'SPO'):
                    #print("SPO")
                    for pairNeurons in range(0, len(neuronList)):
                        # Invert the V of the neurons between the selected pairs of neurons

                        #print("selectedPairsNeurons[pairNeurons][0]", neuronList[pairNeurons][0])
                        #print("selectedPairsNeurons[pairNeurons][1]", neuronList[pairNeurons][1])
                        #print("layerConv1.v[selectedPairsNeurons[pairNeurons][0]]", layerConv1.v[neuronList[pairNeurons][0]])
                        #print("layerConv1.v[selectedPairsNeurons[pairNeurons][1]]", layerConv1.v[neuronList[pairNeurons][1]])

                        tmpV = layerConv1.v[neuronList[pairNeurons][0]]
                        layerConv1.v[neuronList[pairNeurons][0]] = layerConv1.v[neuronList[pairNeurons][1]]
                        layerConv1.v[neuronList[pairNeurons][1]] = tmpV

                        #print("-- AFTER --")
                        #print("layerConv1.v[selectedPairsNeurons[pairNeurons][0]]", layerConv1.v[neuronList[pairNeurons][0]])
                        #print("layerConv1.v[selectedPairsNeurons[pairNeurons][1]]", layerConv1.v[neuronList[pairNeurons][1]])
                        #print("-----------------------------")  
            
                elif (attack == 'SYB'):
                    #print("SYB")
                    # Perform the attack over the selected neurons
                    for neuron in neuronList:  
                        #print("nNeuron: ", neuron)
                        #print("layerConv1.v[neuron]: ", layerConv1.v[neuron])
                        layerConv1.v[neuron] = resetValue*mV + thresholdValue*mV - layerConv1.v[neuron]
                        #print("NEW layerConv1.v[neuron]: ", layerConv1.v[neuron])
                        
            layerConv1.timeCounter[0] += 1*ms #STEP_TIME
            #print("AFTER layer1.timeCounter[0]: ", layer1.timeCounter[0])
            #print("----------------------------")
    
    
    # Invoke method based on the time distance of the sequential attacks
    @network_operation(dt=1*ms)
    def periodicFunctionSequentialAttacks():
        if(isSequentialAttack):
            #print(isSequentialAttack)
            #currentTimeCounter = int(layerConv1.timeCounter[0]*1000)
            
            # SEQUENTIAL ATTACKS (SCA, FOR)
            currentCounter = int(layerConv1.neuronCounter[0])
            
            # Rat starts on (0,0) at STEP_TIME ms
            #if layerConv1.timeCounter[0] >= STEP_TIME:
            
            if (layerConv1.timeCounter[0] >= instantsAttack[0]):
                #print("time: ", layerConv1.timeCounter[0]/ms)
                #print("modulo: ", round(((layerConv1.timeCounter[0]/ms) % (timeStepsSeqAttacks/ms))))
                
                #if round(((layerConv1.timeCounter[0]/ms) % (timeStepsSeqAttacks/ms))) == 0: 
                if trunc(trunc((layerConv1.timeCounter[0]/ms)) % (timeStepsSeqAttacks/ms)) == 0:
                    
                    #print("instante: ", layerConv1.timeCounter[0]/ms)
                    if(currentCounter < 200):
                        #print("counter: ", currentCounter)
                        #print("G.v[counter]: ", layerConv1.v[currentCounter])
                        
                        if(attack=='SCA'):
                            layerConv1.v[currentCounter] += vIncrement*mV
                        else:
                            layerConv1.v[currentCounter] -= vIncrement*mV

                        #print("NEW G.v[counter]: ", layerConv1.v[currentCounter])
                        layerConv1.neuronCounter[0] += 1
                        #print("-------------------------")
                    
            layerConv1.timeCounter[0] += 1*ms
            
            #print("AFTER layer1.timeCounter[0]: ", layer1.timeCounter[0])
            #print("----------------------------")
    
    run(simulationDuration)
    
    return [spikeMonGlobal, stateMonGlobal]


# EXAMPLES FOR TESTING PURPOSES
#spikeMons, stateMons = runNetworkAttacks('SCA', True, 3*ms, None, 1.0, SIMULATION_TIME, [])
#dump_simulation_data_to_csv("SCA", str(1), "X", str("1.0"), str(0), spikeMons[3])

#spikeMons, stateMons = runNetworkAttacks('FOR', True, 3*ms, None, -1, SIMULATION_TIME, [])
#dump_simulation_data_to_csv("FOR", str(1), "X", str("-1.0"), str(0), spikeMons[3])

#listN = [0]

#spikeMons, stateMons = runNetworkAttacks('FLO', False, 1*ms, [25*ms, 50*ms, 75*ms], 1.0, SIMULATION_TIME, listN)
#dump_simulation_data_to_csv("FLO", str(currTest), str(3), str(1), list_neurons_to_string(listN), "(0,0)", str("1.0"), str(0), spikeMons[3])            


#spikeMons, stateMons = runNetworkAttacks('JAM', False, 1*ms, 100*ms, -1, SIMULATION_TIME, [20, 75, 120])
#dump_simulation_data_to_csv("JAM", str(1), "X", str("-1.0"), str(0), spikeMons[3])

#spikeMons, stateMons = runNetworkAttacks('SPO', False, 1*ms, 100*ms, None, SIMULATION_TIME, [[20, 30], [70, 80]])
#dump_simulation_data_to_csv("SPO", str(1), "X", str("-1.0"), str(0), spikeMons[3])

#spikeMons, stateMons = runNetworkAttacks('SYB', False, 1*ms, 100*ms, None, SIMULATION_TIME, [20, 30, 70, 80])
#dump_simulation_data_to_csv("SYB", str(1), "X", str("-1.0"), str(0), spikeMons[3])

### Tests for comparating between different combination of parameters

In [ ]:
csv_header_neurons_global = [
    "test", "n_attacks", "n_neurons", "stim_value", "vIncrement", "paramI",
    "glbl_i_spikes_mean","glbl_a_spikes_mean","glbl_i_spikes_std","glbl_a_spikes_std","glbl_shifts_mean",
    "attk_i_spikes_mean","attk_a_spikes_mean","attk_i_spikes_std","attk_a_spikes_std","attk_shifts_mean",
]

csv_header_neurons_layers = [
    "test", "n_attacks", "n_neurons", "stim_value", "vIncrement", "paramI", "t_window",
    "l1_glbl_i_spikes_mean","l1_glbl_a_spikes_mean","l1_glbl_i_spikes_std","l1_glbl_a_spikes_std","l1_glbl_shifts_mean",
    "l1_attk_i_spikes_mean","l1_attk_a_spikes_mean","l1_attk_i_spikes_std","l1_attk_a_spikes_std","l1_attk_shifts_mean",
    "l2_glbl_i_spikes_mean","l2_glbl_a_spikes_mean","l2_glbl_i_spikes_std","l2_glbl_a_spikes_std","l2_glbl_shifts_mean",
    "l2_attk_i_spikes_mean","l2_attk_a_spikes_mean","l2_attk_i_spikes_std","l2_attk_a_spikes_std","l2_attk_shifts_mean",
    "l3_glbl_i_spikes_mean","l3_glbl_a_spikes_mean","l3_glbl_i_spikes_std","l3_glbl_a_spikes_std","l3_glbl_shifts_mean",
    "l3_attk_i_spikes_mean","l3_attk_a_spikes_mean","l3_attk_i_spikes_std","l3_attk_a_spikes_std","l3_attk_shifts_mean",
]

csv_header_dispersion = [
    "test", "n_attacks", "n_neurons", "stim_value", "vIncrement", "attack", "t_window", "paramI", 
    "distance_instants", "distance_spikes"#, "distance_centroid"
]

# Stats files
stats_dir_FLO = "csv_stats/FLO/"
stats_dir_SCA = "csv_stats/SCA/"
    
csv_filename_FLO = stats_dir_FLO+"temp_export_FLO.csv"
csv_filename_SCA = stats_dir_SCA+"temp_export_SCA.csv"

stats_neurons_global_csv_filename_FLO = stats_dir_FLO+"stats_neurons_global_FLO.csv"
stats_neurons_layers_csv_filename_FLO = stats_dir_FLO+"stats_neurons_layers_FLO.csv"

stats_neurons_global_csv_filename_SCA = stats_dir_SCA+"stats_neurons_global_SCA.csv"
stats_neurons_layers_csv_filename_SCA = stats_dir_SCA+"stats_neurons_layers_SCA.csv"

stats_dispersion_csv_filename_FLO = stats_dir_FLO+"stats_dispersion_FLO.csv"
stats_dispersion_csv_filename_SCA = stats_dir_SCA+"stats_dispersion_SCA.csv"

output_file_FLO = stats_dir_FLO+"output_moving_FLO.txt"
output_file_SCA = stats_dir_SCA+"output_moving_SCA.txt"

# Plots
plots_dir_FLO_raster = "plots/FLO/rasters/"
plots_dir_FLO_spikes = "plots/FLO/spikes/"
plots_dir_FLO_dispersion = "plots/FLO/dispersion/"
plots_dir_FLO_shifts = "plots/FLO/shifts/"

plots_dir_SCA_raster = "plots/SCA/rasters/"
plots_dir_SCA_spikes = "plots/SCA/spikes/"
plots_dir_SCA_dispersion = "plots/SCA/dispersion/"
plots_dir_SCA_shifts = "plots/SCA/shifts/"

# Raster plots
csv_rasters_FLO = stats_dir_FLO+"rasters_FLO.csv"
csv_rasters_SCA = stats_dir_SCA+"rasters_SCA.csv"

# Raster aggregated ("cloud of dots") for each position
aggr_csv_filename_FLO = stats_dir_FLO+"aggr_FLO.csv"
aggr_csv_filename_SCA = stats_dir_SCA+"aggr_SCA.csv"

condition_generate_rasters = {
    "n_neurons": [55],
    "v_increment": [40],
}

T1_init = 0
T1_fin = 1000

T2_init = 12000
T2_fin = 13000

T3_init = 26000
T3_fin = 27000

In [ ]:
# Create directories if they do not exist

if "FLO" in list_attack_generation:    
    # Stats
    if not os.path.exists(stats_dir_FLO):
        os.makedirs(stats_dir_FLO)
        
    # Plots
    if not os.path.exists(plots_dir_FLO_raster):
        os.makedirs(plots_dir_FLO_raster)

    if not os.path.exists(plots_dir_FLO_spikes):
        os.makedirs(plots_dir_FLO_spikes)

    if not os.path.exists(plots_dir_FLO_dispersion):
        os.makedirs(plots_dir_FLO_dispersion)

    if not os.path.exists(plots_dir_FLO_shifts):
        os.makedirs(plots_dir_FLO_shifts)

if "SCA" in list_attack_generation:
    # Stats
    if not os.path.exists(stats_dir_SCA):
        os.makedirs(stats_dir_SCA)

    # Plots
    if not os.path.exists(plots_dir_SCA_raster):
        os.makedirs(plots_dir_SCA_raster)

    if not os.path.exists(plots_dir_SCA_spikes):
        os.makedirs(plots_dir_SCA_spikes)

    if not os.path.exists(plots_dir_SCA_dispersion):
        os.makedirs(plots_dir_SCA_dispersion)

    if not os.path.exists(plots_dir_SCA_shifts):
        os.makedirs(plots_dir_SCA_shifts)

In [ ]:
# Overwrite export files

if "FLO" in list_attack_generation:
    open(csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_FLO, ["attack", "test", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster files
    open(csv_rasters_FLO, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_FLO, ["attack", "test", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
    
    # Overwrite raster aggregated files
    open(aggr_csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_FLO, ["time_delta", "number_spikes", "attack", "position", "n_neurons", "n_exec"])

    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_FLO, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_FLO, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_FLO, csv_header_neurons_layers)
    
    # Output file
    open(output_file_FLO, 'w').close() # Remove file content
    
if "SCA" in list_attack_generation:
    open(csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(csv_filename_SCA, ["attack", "test", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

    # Overwrite raster files
    open(csv_rasters_SCA, 'w').close() # Remove file content
    append_to_csv_file(csv_rasters_SCA, ["attack", "test", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

    # Overwrite raster aggregated files
    open(aggr_csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(aggr_csv_filename_SCA, ["time_delta", "number_spikes", "attack", "position", "n_neurons", "n_exec"])
    
    # Overwrite stats neurons CSV
    open(stats_neurons_global_csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_global_csv_filename_SCA, csv_header_neurons_global)

    open(stats_neurons_layers_csv_filename_SCA, 'w').close() # Remove file content
    append_to_csv_file(stats_neurons_layers_csv_filename_SCA, csv_header_neurons_layers)
    
    # Output file
    open(output_file_SCA, 'w').close() # Remove file content

In [ ]:
def generate_stats_neurons_pandas(test_id, attack, csv_filename, stats_neurons_global_csv_filename, stats_neurons_layers_csv_filename):
    df = pd.read_csv(csv_filename, delimiter=";", dtype={'n_neurons': np.int32})
    
    #### 1) Aggregated statistics for all layers (global)
    data_initial_state = df[df.attack == "initial_state"]
    df_attack = df[df.attack == attack]
    
    list_attacked_neurons = df_attack["attacked_neurons"].unique()[0].split("-")
    
    grouped = df_attack.groupby('neuron').count()["time_delta"]
    d_attack_aggr_neuron = pd.DataFrame()
    d_attack_aggr_neuron['neuron'] = grouped.index
    d_attack_aggr_neuron['n_spikes'] = grouped.tolist()
    
    # Get the data from the initial_state experiment based on the paramI value
    param_I = df_attack.paramI.unique()[0]
    
    grouped = data_initial_state[(data_initial_state.paramI == param_I)].groupby('neuron').count()["time_delta"]
    d_initial_aggr_neuron = pd.DataFrame()
    d_initial_aggr_neuron['neuron'] = grouped.index
    d_initial_aggr_neuron['n_spikes'] = grouped.tolist()
    
    data_csv = []
    data_csv.append(test_id)
    data_csv.append(df_attack["n_attacks"].unique()[0])
    data_csv.append(df_attack["n_neurons"].unique()[0])
    data_csv.append(df_attack["stim_value"].unique()[0])
    data_csv.append(df_attack["vIncrement"].unique()[0])
    data_csv.append(df_attack["paramI"].unique()[0])   
    
    ### GLOBAL PERSPECTIVE
    # Total number of spikes
    initial_nSpikes = data_initial_state[(data_initial_state.paramI == param_I)].count()["neuron"]
    attack_nSpikes = df_attack.count()["neuron"]

    # Mean of spikes (over the total neurons of the layer)
    initial_mean_spikes = round(d_initial_aggr_neuron["n_spikes"].mean(), 2)
    attack_mean_spikes = round(d_attack_aggr_neuron["n_spikes"].mean(), 2)

    # Standard deviation (over the total neurons of the layer)
    initial_std_spikes = round(d_initial_aggr_neuron["n_spikes"].std(), 2)
    attack_std_spikes = round(d_attack_aggr_neuron["n_spikes"].std(), 2)

    # Percentage of shifts between the attack and normal behaviour
    initial_nSpikes_join = data_initial_state[(data_initial_state.paramI == param_I)][["attack", "time_delta", "neuron"]].copy()
    attack_nSpikes_join = df_attack[["attack", "time_delta", "neuron"]].copy()
    join_initial_attack_shifts = pd.merge(attack_nSpikes_join, initial_nSpikes_join,  how='left', left_on=['time_delta','neuron'], right_on = ['time_delta','neuron'])
    join_initial_attack_shifts = join_initial_attack_shifts.fillna(0)
    n_shifts_initial_attack = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0].count()["neuron"]
    percent_shifts_initial_attack = round((n_shifts_initial_attack/attack_nSpikes)*100,2)

    # List of neurons shifted in this layer
    attack_shifted_neurons = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0]["neuron"].unique()

    ### ATTACKED NEURONS PERSPECTIVE

    # spikes: mean
    a_initial_mean_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)
    a_attack_mean_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)

    # Standard deviation
    a_initial_std_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)
    a_attack_std_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)

    # mean of % shifts
    a_join_shifts_attk = join_initial_attack_shifts[join_initial_attack_shifts.neuron.isin(attack_shifted_neurons)]
    a_n_shifts_zeros = a_join_shifts_attk[a_join_shifts_attk.attack_y == 0].count()["neuron"]
    # number of shifts under attack (count zeros) / total number of spikes under attack 
    a_mean_shifts = round((a_n_shifts_zeros/a_join_shifts_attk.neuron.count())*100, 2)

    data_csv.append(initial_mean_spikes)
    data_csv.append(attack_mean_spikes)
    data_csv.append(initial_std_spikes)
    data_csv.append(attack_std_spikes)
    data_csv.append(percent_shifts_initial_attack)

    data_csv.append(a_initial_mean_spikes)
    data_csv.append(a_attack_mean_spikes)
    data_csv.append(a_initial_std_spikes)
    data_csv.append(a_attack_std_spikes)
    data_csv.append(a_mean_shifts)
    
    append_to_csv_file(stats_neurons_global_csv_filename, data_csv)
    
    #### 2) Statistics per each layer ####
    for position in range(1, 4):
        if position == 1:
            min_range_time = T1_init
            max_range_time = T1_fin
        elif position == 2:
            min_range_time = T2_init
            max_range_time = T2_fin
        elif position == 3:
            min_range_time = T3_init
            max_range_time = T3_fin
        
        data_csv = []
        data_csv.append(test_id)
        data_csv.append(df_attack["n_attacks"].unique()[0])
        data_csv.append(df_attack["n_neurons"].unique()[0])
        data_csv.append(df_attack["stim_value"].unique()[0])
        data_csv.append(df_attack["vIncrement"].unique()[0])
        data_csv.append(df_attack["paramI"].unique()[0])
        data_csv.append(position)
        
        # We need to filter by time_delta before calculating statistics        
        data_initial_state = df[(df.attack == "initial_state") & (df.time_delta >= min_range_time) & (df.time_delta < max_range_time)]
        df_attack = df[(df.attack == attack) & (df.time_delta >= min_range_time) & (df.time_delta < max_range_time)]

        list_attacked_neurons = df_attack["attacked_neurons"].unique()[0].split("-")

        grouped = df_attack.groupby('neuron').count()["time_delta"]
        d_attack_aggr_neuron = pd.DataFrame()
        d_attack_aggr_neuron['neuron'] = grouped.index
        d_attack_aggr_neuron['n_spikes'] = grouped.tolist()

        # Get the data from the initial_state experiment based on the paramI value
        param_I = df_attack.paramI.unique()[0]

        grouped = data_initial_state[(data_initial_state.paramI == param_I)].groupby('neuron').count()["time_delta"]
        d_initial_aggr_neuron = pd.DataFrame()
        d_initial_aggr_neuron['neuron'] = grouped.index
        d_initial_aggr_neuron['n_spikes'] = grouped.tolist()
        
        for layer in range(1, 4):
            if layer == 1:
                min_range_neuron = 0
                max_range_neuron = 200
            elif layer == 2:
                min_range_neuron = 200
                max_range_neuron = 272
            elif layer == 3:
                min_range_neuron = 272
                max_range_neuron = 276

            ### GLOBAL PERSPECTIVE
            # Total number of spikes
            initial_nSpikes = data_initial_state[(data_initial_state.paramI == param_I) & (data_initial_state.neuron >= min_range_neuron) & (data_initial_state.neuron < max_range_neuron)].count()["neuron"]
            attack_nSpikes = df_attack[(df_attack.neuron >= min_range_neuron) & (df_attack.neuron < max_range_neuron)].count()["neuron"]

            # Mean of spikes (over the total neurons of the layer)
            initial_mean_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron >= min_range_neuron) & (d_initial_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].mean(), 2)
            attack_mean_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron >= min_range_neuron) & (d_attack_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].mean(), 2)

            # Standard deviation (over the total neurons of the layer)
            initial_std_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron >= min_range_neuron) & (d_initial_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].std(), 2)
            attack_std_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron >= min_range_neuron) & (d_attack_aggr_neuron.neuron < max_range_neuron)]["n_spikes"].std(), 2)

            # Percentage of shifts between the attack and normal behaviour
            initial_nSpikes_join = data_initial_state[(data_initial_state.paramI == param_I) & (data_initial_state.neuron >= min_range_neuron) & (data_initial_state.neuron < max_range_neuron)][["attack", "time_delta", "neuron"]].copy()
            
            attack_nSpikes_join = df_attack[(df_attack.neuron >= min_range_neuron) & (df_attack.neuron < max_range_neuron)][["attack", "time_delta", "neuron"]].copy()
            
            join_initial_attack_shifts = pd.merge(attack_nSpikes_join, initial_nSpikes_join,  how='left', left_on=['time_delta','neuron'], right_on = ['time_delta','neuron'])
            join_initial_attack_shifts = join_initial_attack_shifts.fillna(0)
            n_shifts_initial_attack = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0].count()["neuron"]
            percent_shifts_initial_attack = round((n_shifts_initial_attack/attack_nSpikes)*100,2)

            # List of neurons shifted in this layer
            attack_shifted_neurons = join_initial_attack_shifts[join_initial_attack_shifts.attack_y == 0]["neuron"].unique()

            ### ATTACKED NEURONS PERSPECTIVE

            # spikes: mean
            a_initial_mean_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)
            a_attack_mean_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].mean(), 2)

            # Standard deviation
            a_initial_std_spikes = round(d_initial_aggr_neuron[(d_initial_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)
            a_attack_std_spikes = round(d_attack_aggr_neuron[(d_attack_aggr_neuron.neuron.isin(attack_shifted_neurons))]["n_spikes"].std(), 2)

            # mean of % shifts
            a_join_shifts_attk = join_initial_attack_shifts[join_initial_attack_shifts.neuron.isin(attack_shifted_neurons)]
            a_n_shifts_zeros = a_join_shifts_attk[a_join_shifts_attk.attack_y == 0].count()["neuron"]
            # number of shifts under attack (count zeros) / total number of spikes under attack 
            a_mean_shifts = round((a_n_shifts_zeros/a_join_shifts_attk.neuron.count())*100, 2)
            
            data_csv.append(initial_mean_spikes)
            data_csv.append(attack_mean_spikes)
            data_csv.append(initial_std_spikes)
            data_csv.append(attack_std_spikes)
            data_csv.append(percent_shifts_initial_attack)

            data_csv.append(a_initial_mean_spikes)
            data_csv.append(a_attack_mean_spikes)
            data_csv.append(a_initial_std_spikes)
            data_csv.append(a_attack_std_spikes)
            data_csv.append(a_mean_shifts)

        append_to_csv_file(stats_neurons_layers_csv_filename, data_csv)

In [ ]:
def generate_aggr_data_pandas(test_id, attack, n_exec, vIncrement, n_neurons, flag, csv_filename, aggr_csv_filename):   
    df = pd.read_csv(csv_filename, delimiter=";", dtype={'n_neurons': np.int32})

    data_initial_state = df[df.attack == "initial_state"]
    df_attack = df[(df.attack == attack) & (df.vIncrement == vIncrement)] 

    # Get the data from the initial_state experiment based on the paramI value
    param_I = df_attack.paramI.unique()[0]
    
    for position in range(0, len(list_instant_optimal_path)):
        min_range = list_instant_optimal_path[position]

        if position == len(list_instant_optimal_path)-1: # last position of the list
            max_range = round(SIMULATION_TIME/ms)
        else:
            max_range = list_instant_optimal_path[position+1]

        # Group by time instant: Extraction of time_delta and count number of spikes in each instant
        grouped = data_initial_state[(data_initial_state.paramI == param_I) & (data_initial_state.time_delta >= min_range) & (data_initial_state.time_delta < max_range)].groupby('time_delta').count()["neuron"]
        d_aggr_initial = pd.DataFrame()
        d_aggr_initial['time_delta'] = grouped.index
        d_aggr_initial['number_spikes'] = grouped.tolist() #[item/276 for item in grouped.tolist()]

        grouped = df_attack[(df_attack.paramI == param_I) & (df_attack.time_delta >= min_range) & (df_attack.time_delta < max_range)].groupby('time_delta').count()["neuron"]
        d_aggr_attack = pd.DataFrame()
        d_aggr_attack['time_delta'] = grouped.index
        d_aggr_attack['number_spikes'] = grouped.tolist() #[item/276 for item in grouped.tolist()]
        
        # Dump only if flag == True
        if flag:
            # Dump to CSV
            d_aggr_initial["attack"] = "initial_state"
            d_aggr_initial["position"] = int(min_range/(STEP_TIME/ms)) #Get number of position
            d_aggr_initial["n_neurons"] = 0 # Initial state does not have n_neurons attacked
            d_aggr_initial["n_exec"] = n_exec
            
            # Export both DF to the same CSV file (append)
            d_aggr_initial.to_csv(aggr_csv_filename, mode = "a", index = False, header=False, sep=";")

        d_aggr_attack["attack"] = attack
        d_aggr_attack["position"] = int(min_range/(STEP_TIME/ms)) #Get number of position
        d_aggr_attack["n_neurons"] = n_neurons
        d_aggr_attack["n_exec"] = n_exec

        d_aggr_attack.to_csv(aggr_csv_filename, mode = "a", index = False, header=False, sep=";")

In [ ]:
def attack_FLO(tAttacks, vIncrement_stats, range_nNeurons, list_vIncrement, list_param_I, nExec):
    currTest = 1
    stimValues = [1.0]
    nAttacks = len(tAttacks)
    
    # Flag to only dump the aggr data of the spontaneous behaviour just once (always the same)
    flag_aggr_initial = True
    
    for paramI in list_param_I:
        ###  Spontaneous simulation (only one per paramI)
        spikeMon, stateMon = spontaneous_simulation(SIMULATION_TIME, paramI)

        # Dump the same data in a persistent file to generate raster plots after simulation
        dump_simulation_data_to_csv("initial_state", str(currTest), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_rasters_FLO) 
        
        # Print progress
        append_to_csv_file(output_file_FLO, ["Test %s: spontaneous simulation | paramI: %s " % (currTest, paramI)])
        
        for nNeurons in range_nNeurons:
            for stimValue in stimValues:
                for vIncrement in list_vIncrement:
                    for currExec in range(0,nExec):
                        # Get a random list of neurons to attack
                        listNeurons = generate_list_random_neurons(nNeurons)

                        # Simulate attack
                        spikeMon_a, stateMon_a = attack_simulation('FLO', False, 1*ms, tAttacks, stimValue, SIMULATION_TIME, listNeurons, vIncrement, paramI)
                        
                        # Store the initial_state data and attack data (needed for BOT neuron and dispersion stats)
                        dump_simulation_data_to_csv("initial_state", str(currTest), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_FLO) 
                        dump_simulation_data_to_csv("FLO", str(currTest), str(len(tAttacks)), str(len(listNeurons)), list_neurons_to_string(listNeurons), "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_filename_FLO)

                        # Dump the same data in a persistent file to generate raster plots after simulation
                        # Only get the data for the first execution of the attack
                        if (vIncrement in condition_generate_rasters["v_increment"] and nNeurons in 
                        condition_generate_rasters["n_neurons"] and currExec == 0): 
                            dump_simulation_data_to_csv("FLO", str(currTest), str(len(tAttacks)), str(len(listNeurons)), list_neurons_to_string(listNeurons), "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_rasters_FLO)
                        
                        # Generate dispersion stats (only for the vIncrement desired)
                        #if vIncrement_stats == vIncrement:
                        #    generate_stats_dispersion_pandas(currTest, "FLO", vIncrement_stats, nNeurons, currExec, csv_filename_FLO, stats_dispersion_csv_filename_FLO, raster_aggr_csv_filename_FLO)
                        
                        # Generate dispersion stats (only for the first execution -> avoid duplicates)
                        if vIncrement_stats == vIncrement: #and currExec == 0:
                            generate_aggr_data_pandas(currTest, "FLO", currExec, vIncrement, nNeurons, flag_aggr_initial, csv_filename_FLO, aggr_csv_filename_FLO)
                            
                            flag_aggr_initial = False
                            
                        # Generate neuron stats
                        generate_stats_neurons_pandas(currTest, "FLO",csv_filename_FLO, stats_neurons_global_csv_filename_FLO, stats_neurons_layers_csv_filename_FLO)

                        # Print progress
                        append_to_csv_file(output_file_FLO, ["Test %s: | paramI: %s | nNeurons attacked: %s | nAttacks: %s | tAttacks: %s | stimValue: %s | vIncrement: %s " % (currTest, paramI, nNeurons, nAttacks, str(tAttacks/ms), stimValue, vIncrement)])

                        currTest += 1

                        # Overwrite export file for next execution
                        open(csv_filename_FLO, 'w').close() # Remove file content
                        append_to_csv_file(csv_filename_FLO, ["attack", "test", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])
        

In [ ]:
import time

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print("Init: ", current_time)

if "FLO" in list_attack_generation:
    #attack_FLO(tAttacks=[50*ms], vIncrement_stats=40, range_nNeurons=range(5, 110, 10), list_vIncrement=[10,20,40,60], list_param_I=[10], nExec=10)
    #attack_FLO(tAttacks=[50*ms], vIncrement_stats=40, range_nNeurons=range(5, 110, 10), list_vIncrement=[40], list_param_I=[10], nExec=2)
    attack_FLO(tAttacks=[50*ms], vIncrement_stats=40, range_nNeurons=range(5, 110, 10), list_vIncrement=[10,20,40,60], list_param_I=[10], nExec=2)

    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print("Fin: ", current_time)

In [ ]:
def attack_SCA(tAttacks, vIncrement_stats, list_vIncrement, list_param_I, nExec):
    currTest = 1
    stimValues = [1.0]
    nAttacks = 1
    listNeurons = [0,1] # dummy data
    
    flag_aggr_initial = True
    
    for paramI in list_param_I:
        ###  Spontaneous simulation (only one per paramI)
        spikeMon, stateMon = spontaneous_simulation(SIMULATION_TIME, paramI)

        # Dump the same data in a persistent file to generate raster plots after simulation
        dump_simulation_data_to_csv("initial_state", str(currTest), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_rasters_SCA) 
        
        # Print progress
        append_to_csv_file(output_file_SCA, ["Test %s: spontaneous simulation | paramI: %s " % (currTest, paramI)])

        for stimValue in stimValues:
            for vIncrement in list_vIncrement:
                for currExec in range(0,nExec):
                    # Simulate attack
                    spikeMon_a, stateMon_a = attack_simulation('SCA', True, get_time_steps_sequential(tAttacks[0]), tAttacks, stimValue, SIMULATION_TIME, [], vIncrement, paramI)
                    
                    # Store the initial_state data and attack data (needed for BOT neuron and dispersion stats)
                    dump_simulation_data_to_csv("initial_state", str(currTest), "0", "0", "X", "X", "0", "0", "0", str(paramI), spikeMon, csv_filename_SCA) 
                    dump_simulation_data_to_csv("SCA", str(currTest), str(len(tAttacks)), str(len([listNeurons])), list_neurons_to_string(listNeurons), "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_filename_SCA)

                    # Dump the same data in a persistent file to generate raster plots after simulation
                    # Only get the data for the first execution of the attack
                    if (vIncrement in condition_generate_rasters["v_increment"]): 
                        dump_simulation_data_to_csv("SCA", str(currTest), str(len(tAttacks)), str(len([listNeurons])), list_neurons_to_string(listNeurons), "(0,0)", str(stimValue), str(currExec), str(vIncrement), str(paramI), spikeMon_a, csv_rasters_SCA)

                    # Generate dispersion stats (only for the vIncrement desired)
                    if vIncrement_stats == vIncrement:
                        generate_aggr_data_pandas(currTest, "SCA", currExec, vIncrement, 0, flag_aggr_initial, csv_filename_SCA, aggr_csv_filename_SCA)
                            
                    #generate_stats_dispersion_pandas(currTest, "SCA", vIncrement_stats, 0, currExec, csv_filename_SCA, stats_dispersion_csv_filename_SCA, raster_aggr_csv_filename_SCA)
                        
                    # Generate neuron stats
                    generate_stats_neurons_pandas(currTest, "SCA", csv_filename_SCA, stats_neurons_global_csv_filename_SCA, stats_neurons_layers_csv_filename_SCA)

                    # Print progress
                    append_to_csv_file(output_file_SCA, ["Test %s: | paramI: %s | nAttacks: %s | tAttacks: %s | stimValue: %s | vIncrement: %s " % (currTest, paramI, nAttacks, str(tAttacks/ms), stimValue, vIncrement)])

                    currTest += 1

                    # Overwrite export file for next execution
                    open(csv_filename_SCA, 'w').close() # Remove file content
                    append_to_csv_file(csv_filename_SCA, ["attack", "test", "n_attacks", "n_neurons", "attacked_neurons", "coord_attack", "stim_value", "n_exec", "vIncrement", "paramI", "time_delta", "neuron"])

In [ ]:
import time

t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print("Init: ", current_time)

if "SCA" in list_attack_generation:
    attack_SCA(tAttacks=[50*ms], vIncrement_stats=40, list_vIncrement=range(5, 65, 5), list_param_I=[10], nExec=1)

    t = time.localtime()
    current_time = time.strftime("%H:%M:%S", t)
    print("Fin: ", current_time)

## Generate plots with the previous statistics

In [ ]:
plt.rcParams.update({'font.family': 'monospace'})
sns.set(style="whitegrid",font_scale=2.5, rc={'figure.figsize':(40,20)})

#kwargs  =   {'edgecolor':"black", # for edge color
#             'linewidth':2, # line width of spot
#             'linestyle':'-', # line style of spot
#            }

plt.rcParams['axes.edgecolor'] = "black" #set the value globally

#sns.set(font_scale=1.4)
#sns.set_style("whitegrid")

In [ ]:
attack = "SCA"

if attack in ["FLO"]:
    csv_rasters = csv_rasters_FLO
    csv_rasters_aggr = aggr_csv_filename_FLO
    stats_neurons_global_filename = stats_neurons_global_csv_filename_FLO
    stats_neurons_layers_filename = stats_neurons_layers_csv_filename_FLO
    #stats_dispersion_filename = stats_dispersion_csv_filename_FLO
    plots_prefix_raster = plots_dir_FLO_raster
    plots_prefix_spikes = plots_dir_FLO_spikes
    plots_prefix_dispersion = plots_dir_FLO_dispersion
    plots_prefix_shifts = plots_dir_FLO_shifts
    x_plot = "n_neurons"
    n_exec = 0

elif attack in ["SCA"]:
    csv_rasters = csv_rasters_SCA
    csv_rasters_aggr = aggr_csv_filename_SCA
    stats_neurons_global_filename = stats_neurons_global_csv_filename_SCA
    stats_neurons_layers_filename = stats_neurons_layers_csv_filename_SCA
    #stats_dispersion_filename = stats_dispersion_csv_filename_SCA
    plots_prefix_raster = plots_dir_SCA_raster
    plots_prefix_spikes = plots_dir_SCA_spikes
    plots_prefix_dispersion = plots_dir_SCA_dispersion
    plots_prefix_shifts = plots_dir_SCA_shifts
    x_plot = "vIncrement"
    n_exec = 0

### 1. Raster plots
> 3x3 grid of plots \
> a) Rows: 3 experiments: 1) spontaneous 2) attack config X. 2) attack config Y. \
> b) Columns: each one of the 3 positions considered. However, we only represent a small subset of the duration of the position (first 1/3 of the position duration)

In [ ]:
df = pd.read_csv(csv_rasters, delimiter=";", dtype={'n_neurons': np.int32})

In [ ]:
list_tests = df.test.unique()

In [ ]:
list_tests

In [ ]:
fraction_position = 10

#### Spontaneous (all duration of simulation)

In [ ]:
'''
fig,ax = plt.subplots()
sns.scatterplot(x='time_delta', y='neuron', data=df[(df.attack == "initial_state")],
                hue='attack', palette=["#092C48"], s=80) #, style='dataset'

#for a in range(min_range, round(max_range)+2):
#    plt.axvline(x=a, color="red")

cont = 1000
for a in range(0, 28):
    plt.axvline(x=cont*a, color="red")

plt.savefig(plots_prefix_raster+"_spontaneous_all_simulation.pdf")
plt.close()

ax.xaxis.set_major_locator(ticker.MultipleLocator(cont))
'''

#### Spontaneous

In [ ]:
for position in range(1, 4):
    if position == 1:
        min_range = T1_init
        max_range = T1_init+((T1_fin-T1_init)/fraction_position)
    elif position == 2:
        min_range = T2_init
        max_range = T2_init+((T2_fin-T2_init)/fraction_position)
    elif position == 3:
        min_range = T3_init
        max_range = T3_init+((T3_fin-T3_init)/fraction_position)
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='neuron', data=df[(df.attack == "initial_state") & (df.time_delta >= min_range) & (df.time_delta < max_range)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'

    #for a in range(min_range, round(max_range)+2):
    #    plt.axvline(x=a, color="red")
        
    plt.savefig(plots_prefix_raster+"P"+str(position)+"_spontaneous.pdf")
    plt.close()
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(25))

#### Attack

In [ ]:
for position in range(1, 4):
    if position == 1:
        min_range = T1_init
        max_range = T1_init+((T1_fin-T1_init)/fraction_position)
    elif position == 2:
        min_range = T2_init
        max_range = T2_init+((T2_fin-T2_init)/fraction_position)
    elif position == 3:
        min_range = T3_init
        max_range = T3_init+((T3_fin-T3_init)/fraction_position)
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='neuron', data=df[(df.attack == attack) & (df.time_delta >= min_range) & (df.time_delta < max_range) & (df.test == list_tests[1])],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'

    #for a in range(min_range, round(max_range)+2):
    #    plt.axvline(x=a, color="red")
    
    plt.savefig(plots_prefix_raster+"P"+str(position)+"_attack.pdf")
    plt.close()
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(25))

### Grid 3x3 Rasters

In [ ]:
df_FLO = pd.read_csv(csv_rasters_FLO, delimiter=";", dtype={'n_neurons': np.int32})
df_SCA = pd.read_csv(csv_rasters_SCA, delimiter=";", dtype={'n_neurons': np.int32})

list_tests_FLO = df_FLO.test.unique()
list_tests_SCA = df_SCA.test.unique()

x_plot_grid = "time_delta"
y_plot_grid = "neuron"

In [ ]:
list_tests_SCA

In [ ]:
fig,axs = plt.subplots(nrows=3, ncols=3, sharey='row', sharex='col', figsize=(80,40))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)

position_text = ""

fontsize_label = 45
ticks_size = 40

# Initial state
for position in range(1, 4):
    if position == 1:
        position_text = "Position 1"
        min_range = T1_init
        max_range = T1_init+((T1_fin-T1_init)/fraction_position)
    elif position == 2:
        position_text = "Position 13"
        min_range = T2_init
        max_range = T2_init+((T2_fin-T2_init)/fraction_position)
    elif position == 3:
        position_text = "Position 27"
        min_range = T3_init
        max_range = T3_init+((T3_fin-T3_init)/fraction_position)
    
    axs_row = axs[0]
    
    sns.scatterplot(x=x_plot_grid, y=y_plot_grid, data=df_FLO[(df_FLO.attack == "initial_state") & (df_FLO.time_delta >= min_range) & (df_FLO.time_delta < max_range)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs_row[position-1]) #, style='dataset'
    
    axs_row[position-1].set_ylabel("Neuron index - Spontaneous", fontsize=fontsize_label, fontweight="bold")
    axs_row[position-1].set_xlabel("Time duration (ms) - "+ position_text, fontsize=fontsize_label, fontweight="bold")
    
    axs_row[position-1].tick_params(labelsize=ticks_size)
    
    axs_row[position-1].get_legend().remove()
    
# FLO
for position in range(1, 4):
    if position == 1:
        position_text = "Position 1"
        min_range = T1_init
        max_range = T1_init+((T1_fin-T1_init)/fraction_position)
    elif position == 2:
        position_text = "Position 13"
        min_range = T2_init
        max_range = T2_init+((T2_fin-T2_init)/fraction_position)
    elif position == 3:
        position_text = "Position 27"
        min_range = T3_init
        max_range = T3_init+((T3_fin-T3_init)/fraction_position)
    
    axs_row = axs[1]
    
    sns.scatterplot(x=x_plot_grid, y=y_plot_grid, data=df_FLO[(df_FLO.attack == "FLO") & (df_FLO.time_delta >= min_range) & (df_FLO.time_delta < max_range) & (df_FLO.test == list_tests_FLO[1])],
                    hue='attack', palette=["#092C48"], s=80, ax=axs_row[position-1]) #, style='dataset' 
    
    axs_row[position-1].set_ylabel("Neuron index - FLO", fontsize=fontsize_label, fontweight="bold")
    axs_row[position-1].set_xlabel("Time duration (ms) - "+ position_text, fontsize=fontsize_label, fontweight="bold")
    
    axs_row[position-1].tick_params(labelsize=ticks_size)
    
    axs_row[position-1].get_legend().remove()
    
# SCA
for position in range(1, 4):
    if position == 1:
        position_text = "Position 1"
        min_range = T1_init
        max_range = T1_init+((T1_fin-T1_init)/fraction_position)
    elif position == 2:
        position_text = "Position 13"
        min_range = T2_init
        max_range = T2_init+((T2_fin-T2_init)/fraction_position)
    elif position == 3:
        position_text = "Position 27"
        min_range = T3_init
        max_range = T3_init+((T3_fin-T3_init)/fraction_position)
    
    axs_row = axs[2]
    
    sns.scatterplot(x=x_plot_grid, y=y_plot_grid, data=df_SCA[(df_SCA.attack == "SCA") & (df_SCA.time_delta >= min_range) & (df_SCA.time_delta < max_range) & (df_SCA.test == list_tests_SCA[1])],
                    hue='attack', palette=["#092C48"], s=80, ax=axs_row[position-1]) #, style='dataset' 
    
    axs_row[position-1].set_ylabel("Neuron index - SCA", fontsize=fontsize_label, fontweight="bold")
    axs_row[position-1].set_xlabel("Time duration (ms) - "+ position_text, fontsize=fontsize_label, fontweight="bold")
    
    axs_row[position-1].tick_params(labelsize=ticks_size)
    
    axs_row[position-1].get_legend().remove()
    
plt.savefig(plots_dir_FLO_raster+"grid_3x3_rasters.pdf")
plt.close()    

### 2. Metric 1: mean of spikes

### 2.1. Metric 1: mean of spikes (global - unify all 3 layers)
> One single plot representing the duration of the complete simulation

In [ ]:
plots_prefix_spikes_global = plots_prefix_spikes+"global/"
    
if not os.path.exists(plots_prefix_spikes_global):
    os.makedirs(plots_prefix_spikes_global)

In [ ]:
df_stats_global = pd.read_csv(stats_neurons_global_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()

In [ ]:
counter = 1

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
label_size = 50
ticks_size = 45
legend_size = 40
line_size = 20

for paramI in list_paramI:
    fig, ax = plt.subplots(figsize=(40,20))

    ax = sns.lineplot(x=x_plot, y="glbl_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=line_size, label="Spontaneous")

    if attack in ["FLO"]:
        for incr in list_vIncrement:
            ax = sns.lineplot(x=x_plot, y="glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.vIncrement == incr)], linewidth=line_size, label="FLO " + str(incr) + "mV")

        ax.set_xlabel("Number of attacked neurons",fontsize=label_size, fontweight='bold')
        
        # Ranges 
        ax.set_xlim(5,105)
        ax.set_xticks(range(5,110, 5))
            
    elif attack in ["SCA"]:
        ax = sns.lineplot(x=x_plot, y="glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=line_size, label="SCA")

        ax.set_xlabel("Voltage increment (mV)",fontsize=label_size, fontweight='bold')

        # Ranges 
        ax.set_xlim(5,60)
        ax.set_xticks(range(5,65, 5))
    
    ax.set_ylabel("Mean spikes",fontsize=label_size, fontweight='bold')

    ax.tick_params(labelsize=ticks_size)
    leg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)        
    leg.get_frame().set_edgecolor('black')
    
    
            
    fig.savefig(plots_prefix_spikes_global+str(counter)+" Global_mean_spikes_global_perspective.pdf")
    plt.close()
    
    counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    fig, ax = plt.subplots(1, 1)

    ax = sns.lineplot(x=x_plot, y="attk_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=10, label="Spontaneous")

    if attack in ["FLO"]:
        for incr in list_vIncrement:
            ax = sns.lineplot(x=x_plot, y="attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

        ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

        cont = 5
        for a in range(0, 22):
            plt.axvline(x=cont*a, color="red")

    elif attack in ["SCA"]:
        ax = sns.lineplot(x=x_plot, y="attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=10, label="Attack")

        ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

        cont = 5
        for a in range(0, 13):
            plt.axvline(x=cont*a, color="red")

    ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

    ax.axes.set_title("Global (unification all layers) | " + attack + " | Mean spikes - attacked neurons perspective",fontsize=50) 

    ax.tick_params(labelsize=30)
    eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

    ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

    fig.savefig(plots_prefix_spikes_global+str(counter)+" Global_mean_spikes_attacked_perspective.pdf")
    plt.close()

    counter+=1
'''

### 3.2. Metric 3: mean of spikes (layers)
> 3x3 grid of plots \
> **Rows**: each layer \
> **Columns**: each one of the 3 positions considered

In [ ]:
plots_prefix_spikes_layers = plots_prefix_spikes+"layers/"

if not os.path.exists(plots_prefix_spikes_layers):
    os.makedirs(plots_prefix_spikes_layers)

In [ ]:
df_stats_global = pd.read_csv(stats_neurons_layers_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()

In [ ]:
counter = 1

#### Layer 1

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        ax = sns.lineplot(x=x_plot, y="l1_glbl_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Spontaneous")
        
        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l1_glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l1_glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 1 | Position " + str(position) + " | " + attack + " | Mean spikes - global perspective",fontsize=50) 
                          
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_spikes_layers+str(counter)+" L1_P"+str(position)+"_mean_spikes_global_perspective.pdf")
        plt.close()

        counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        ax = sns.lineplot(x=x_plot, y="l1_attk_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Spontaneous")

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l1_attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l1_attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 1 | Position " + str(position) + " | " + attack + " | Mean spikes - attacked neurons perspective",fontsize=50) 

        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_spikes_layers+str(counter)+" L1_P"+str(position)+"_mean_spikes_attacked_perspective.pdf")
        plt.close()

        counter+=1
'''

#### Layer 2

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        ax = sns.lineplot(x=x_plot, y="l2_glbl_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Spontaneous")

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l2_glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l2_glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 2 | Position " + str(position) + " | " + attack + " | Mean spikes - global perspective",fontsize=50) 
                          
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_spikes_layers+str(counter)+" L2_P"+str(position)+"_mean_spikes_global_perspective.pdf")
        plt.close()

        counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        ax = sns.lineplot(x=x_plot, y="l2_attk_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Spontaneous")

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l2_attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l2_attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 2 | Position " + str(position) + " | " + attack + " | Mean spikes - attacked neurons perspective",fontsize=50) 
        
        
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_spikes_layers+str(counter)+" L2_P"+str(position)+"_mean_spikes_attacked_perspective.pdf")
        plt.close()

        counter+=1
'''

#### Layer 3

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        ax = sns.lineplot(x=x_plot, y="l3_glbl_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Spontaneous")

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l3_glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l3_glbl_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 3 | Position " + str(position) + " | " + attack + " | Mean spikes - global perspective",fontsize=50) 
                          
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        
        
        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_spikes_layers+str(counter)+" L3_P"+str(position)+"_mean_spikes_global_perspective.pdf")
        plt.close()

        counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        ax = sns.lineplot(x=x_plot, y="l3_attk_i_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Spontaneous")

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l3_attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l3_attk_a_spikes_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 3 | Position " + str(position) + " | " + attack + " | Mean spikes - attacked neurons perspective",fontsize=50) 
        
        
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_spikes_layers+str(counter)+" L3_P"+str(position)+"_mean_spikes_attacked_perspective.pdf")
        plt.close()

        counter+=1
'''

### Grid 3x3 Spikes

In [ ]:
plots_prefix_spikes_layers = plots_prefix_spikes+"layers/"

if not os.path.exists(plots_prefix_spikes_layers):
    os.makedirs(plots_prefix_spikes_layers)
    
df_stats_global = pd.read_csv(stats_neurons_layers_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()    

In [ ]:
fig,axs = plt.subplots(nrows=3, ncols=3, sharey='row', sharex='col', figsize=(80,40))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)

position_text = ""

fontsize_label = 40
ticks_size = 30

y_plot_initial = ""
y_plot_attack = ""

for layer in range(1,4):
    axs_row = axs[layer-1]
    
    if layer == 1:
        y_plot_initial = "l1_glbl_i_spikes_mean"
        y_plot_attack = "l1_glbl_a_spikes_mean"
    elif layer == 2:
        y_plot_initial = "l2_glbl_i_spikes_mean"
        y_plot_attack = "l2_glbl_a_spikes_mean"
    elif layer == 3:
        y_plot_initial = "l3_glbl_i_spikes_mean"
        y_plot_attack = "l3_glbl_a_spikes_mean"
      
    for position in range(1, 4):
        if position == 1:
            position_text = "Position 1"
        elif position == 2:
            position_text = "Position 13"
        elif position == 3:
            position_text = "Position 27"
        
        sns.lineplot(x=x_plot, y=y_plot_initial, data=df_stats[(df_stats.t_window == position)], linewidth=10, label="Spontaneous", ax=axs_row[position-1])
        
        if attack in ["FLO"]:
            for incr in list_vIncrement:
                sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr), ax=axs_row[position-1])

            cont = 5
            for a in range(0, 22):
                axs_row[position-1].axvline(x=cont*a, color="red")
            
            axs_row[position-1].set_xlabel(position_text+" - Number of attacked neurons", fontsize=fontsize_label, fontweight="bold")

        elif attack in ["SCA"]:
            sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position)], linewidth=10, label="Attack", ax=axs_row[position-1])

            cont = 5
            for a in range(0, 13):
                axs_row[position-1].axvline(x=cont*a, color="red")
            
            axs_row[position-1].set_xlabel(position_text+" - Voltage increment", fontsize=fontsize_label, fontweight="bold")

        if layer!=1 or position !=1 :
            axs_row[position-1].get_legend().remove()
        
        axs_row[position-1].set_ylabel("Layer "+str(layer)+" - Mean spikes", fontsize=fontsize_label, fontweight="bold")
        axs_row[position-1].tick_params(labelsize=ticks_size)
        
fig.savefig(plots_prefix_spikes_layers+"grid_3_3_spikes.pdf")
plt.close()

### Grid 3x1 Spikes

In [ ]:
plots_prefix_spikes_layers = plots_prefix_spikes+"layers/"

if not os.path.exists(plots_prefix_spikes_layers):
    os.makedirs(plots_prefix_spikes_layers)
    
df_stats_global = pd.read_csv(stats_neurons_layers_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()    

In [ ]:
label_size = 50
ticks_size = 45
legend_size = 40
line_size = 20

fig,axs = plt.subplots(nrows=3, ncols=1, sharey='row', sharex='col', figsize=(40,30))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.04)

position_text = ""

y_plot_initial = ""
y_plot_attack = ""

for layer in range(1,4):
    
    if layer == 1:
        y_plot_initial = "l1_glbl_i_spikes_mean"
        y_plot_attack = "l1_glbl_a_spikes_mean"
    elif layer == 2:
        y_plot_initial = "l2_glbl_i_spikes_mean"
        y_plot_attack = "l2_glbl_a_spikes_mean"
    elif layer == 3:
        y_plot_initial = "l3_glbl_i_spikes_mean"
        y_plot_attack = "l3_glbl_a_spikes_mean"
      
    for position in [3]: #range(1, 4):
        if position == 1:
            position_text = "Position 1"
        elif position == 2:
            position_text = "Position 13"
        elif position == 3:
            position_text = "Position 27"
        
        sns.lineplot(x=x_plot, y=y_plot_initial, data=df_stats[(df_stats.t_window == position)], linewidth=line_size, label="Spontaneous", ax=axs[layer-1])
        
        if attack in ["FLO"]:
            for incr in list_vIncrement:
                sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=line_size, label="FLO " + str(incr) + "mV", ax=axs[layer-1])
            
            axs[layer-1].set_xlabel("Number of attacked neurons", fontsize=label_size, fontweight="bold")
            
            # Ranges 
            axs[layer-1].set_xlim(5,105)
            axs[layer-1].set_xticks(range(5,110, 5))
            
        elif attack in ["SCA"]:
            sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position)], linewidth=line_size, label="Attack", ax=axs[layer-1])

            axs[layer-1].set_xlabel("Voltage increment (mV)", fontsize=label_size, fontweight="bold")
            
            # Ranges 
            axs[layer-1].set_xlim(5,60)
            axs[layer-1].set_xticks(range(5,65, 5))
        
        axs[layer-1].set_ylabel("Mean spikes (L"+str(layer)+")", fontsize=label_size, fontweight="bold")
        axs[layer-1].tick_params(labelsize=ticks_size)
        axs[layer-1].legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)
        axs[layer-1].get_legend().get_frame().set_edgecolor('black')
        
        if layer!=1:
            axs[layer-1].get_legend().remove()
            
fig.savefig(plots_prefix_spikes_layers+"grid_1_3_spikes.pdf")
plt.close()

### 3. Metric 2: vertical & horizontal dispersion per position (+ total number of spikes per position)

In [ ]:
if not os.path.exists(plots_prefix_dispersion):
    os.makedirs(plots_prefix_dispersion)

In [ ]:
csv_rasters_aggr = "/data/ahuertas/Brian/csv_stats/IEEE_Access/SCA/aggr_SCA.csv"

In [ ]:
df_aggr = pd.read_csv(csv_rasters_aggr, delimiter=";")
list_n_neurons = df_aggr["n_neurons"].unique().tolist()
list_positions = df_aggr["position"].unique().tolist()

#### Total number of spikes per position

In [ ]:
df_total_spikes = df_aggr.copy()

In [ ]:
df_total_spikes["attack_n_neurons"] = df_total_spikes["attack"].astype("str") + " " + df_total_spikes["n_neurons"].astype("str")

In [ ]:
df_total_spikes = pd.DataFrame(df_total_spikes.groupby(["attack_n_neurons", "position", "n_exec"])["number_spikes"].sum())

In [ ]:
df_total_spikes.reset_index(inplace=True)

In [ ]:
# Start in position 1, not 0
df_total_spikes.position = df_total_spikes.position+1

#### Number of neurons related to visible positions, for each position

In [ ]:
d = {'neurons': np.array(list(dict_all_neurons_count.values())), 'position': np.arange(0,27)}
df_visible = pd.DataFrame(d)

In [ ]:
df_visible.position = df_visible.position+1 

In [ ]:
type(ticker.MultipleLocator(1))

In [ ]:
label_size = 50
ticks_size = 45
legend_size = 40
line_size = 20

fig,ax = plt.subplots(figsize=(40,20))
sns.lineplot(x="position", y="neurons", data=df_visible, linewidth=line_size)

ax.set_xlabel("Position", fontsize=label_size, fontweight='bold')
ax.set_ylabel("Number of visible neurons", fontsize=label_size, fontweight='bold')
ax.tick_params(labelsize=ticks_size)

ax.xaxis.set_major_locator(ticker.IndexLocator(1, 0))

ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)

plt.savefig(plots_prefix_dispersion+"visible_neurons.pdf")
plt.close()

#### Horizontal dispersion

In [ ]:
df_horizontal = df_aggr.copy()

In [ ]:
df_horizontal["attack_n_neurons"] = df_horizontal["attack"].astype("str") + " " + df_horizontal["n_neurons"].astype("str")

In [ ]:
df_horizontal = pd.DataFrame(df_horizontal.groupby(["attack_n_neurons", "position", "n_exec"])["time_delta"].count()/10000*100)

In [ ]:
df_horizontal.reset_index(inplace=True)

In [ ]:
# Start in position 1, not 0
df_horizontal.position = df_horizontal.position+1

In [ ]:
label_size = 50
ticks_size = 45
legend_size = 40
line_size = 20

fig, ax = plt.subplots(figsize=(40,20))

if attack == "FLO":
    sns.pointplot(x="position", y="time_delta", data=df_horizontal[df_horizontal.attack_n_neurons.isin(["initial_state 0", "FLO 55", "FLO 105"])], hue="attack_n_neurons", scale=3.2, errwidth=12, ax=ax)

    # ALL n_neurons
    #sns.pointplot(x="position", y="time_delta", data=df_horizontal[df_horizontal.attack_n_neurons.isin(["initial_state 0", "FLO 55", "FLO 105"])], hue="attack_n_neurons", scale=3.2, errwidth=12, ax=ax)

if attack == "SCA":
    sns.pointplot(x="position", y="time_delta", data=df_horizontal, hue="attack_n_neurons", scale=3.2, ax=ax)

ax.set_xlabel("Position", fontsize=label_size, fontweight="bold")
ax.set_ylabel("Percentage of instants with spikes", fontsize=label_size, fontweight="bold")
ax.tick_params(labelsize=ticks_size)

ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)

leg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)  
leg.get_frame().set_edgecolor('black')

if attack == "FLO":
    leg.get_texts()[0].set_text("FLO 105 neurons")
    leg.get_texts()[1].set_text("FLO 55 neurons")
    leg.get_texts()[2].set_text("Spontaneous")

if attack == "SCA":
    leg.get_texts()[0].set_text("SCA")
    leg.get_texts()[1].set_text("Spontaneous")

fig.savefig(plots_prefix_dispersion+"horizontal_dispersion.pdf")
plt.close()

In [ ]:
'''
# Number of spikes per position
fig, ax = plt.subplots(1,1)

# Initial state
sns.lineplot(data=df_aggr[(df_aggr.attack == "initial_state")].groupby("position")["time_delta"].count()/10000*100, linewidth=10, label="Spontaneous", ax=ax)

# Attack
for n_neurons in [105]:#in list_n_neurons:
    sns.lineplot(data=df_aggr[(df_aggr.attack == attack) & (df_aggr.n_neurons == n_neurons) & (df_aggr.n_exec == 0)].groupby("position")["time_delta"].count()/10000*100, linewidth=10, label=attack+" - "+str(n_neurons), ax=ax)


cont = 5
for a in list_positions:
    ax.axvline(x=a, color="red")

ax.set_xlabel("Positions")
ax.set_ylabel("Percentage of instants with spikes")

#fig.savefig(plots_prefix_dispersion+"horizontal_dispersion.pdf")
#plt.close()
'''

#### Vertical dispersion

In [ ]:
df_vertical = df_aggr.copy()

In [ ]:
df_vertical.position = df_vertical.position+1

In [ ]:
df_vertical

In [ ]:
label_size = 30
ticks_size = 25
legend_size = 20
line_size = 10

def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    counter = 0
    for art in ax.get_children():
        if isinstance(art, PolyCollection):
            if counter % 2 == 0:
                art.set_edgecolor("#4C72B0")
            else:
                art.set_edgecolor("#DD8452")
        
            counter+=1

# Conditions to plot

n_neurons = 105
n_exec = 0

fig, ax = plt.subplots(figsize=(40,20))

if attack == "FLO":
    ax = sns.violinplot(x="position", y="number_spikes", hue="attack", cut=0, 
                        data=df_vertical[(df_vertical.attack == "initial_state") | 
                                     ((df_vertical.attack == "FLO") & (df_vertical.n_neurons == n_neurons) & 
                                      (df_vertical.n_exec == n_exec))]) # split=True

if attack == "SCA":
    ax = sns.violinplot(x="position", y="number_spikes", hue="attack", cut=0, data=df_vertical) # split=True
    
patch_violinplot()


ax.set_xlabel("Position", fontsize=label_size, fontweight="bold")
ax.set_ylabel("Number of spikes", fontsize=label_size, fontweight="bold")
ax.tick_params(labelsize=ticks_size)

ax.grid(b=True, which='major', color='#d8dcd6', linewidth=1.0)
ax.grid(b=True, which='minor', color='#d8dcd6', linewidth=0.5)

leg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)  
leg.get_frame().set_edgecolor('black')

if attack == "FLO":
    leg.get_texts()[0].set_text("Spontaneous")
    leg.get_texts()[1].set_text("FLO 105 neurons")

if attack == "SCA":
    leg.get_texts()[0].set_text("Spontaneous")
    leg.get_texts()[1].set_text("SCA")


fig.savefig(plots_prefix_dispersion+"vertical_dispersion_"+str(n_exec)+".pdf")
plt.close()

### 4. Raster plot aggregation (n_spikes per instant) per position

In [ ]:
df = pd.read_csv(csv_rasters_aggr, delimiter=";", dtype={'n_neurons': np.int32})

In [ ]:
df

In [ ]:
df[(df.attack == "initial_state") & (df.position == 0)]

In [ ]:
positions_rasters_aggr = [0, 12, 26]

In [ ]:
# Spontaneous
'''
for position in positions_rasters_aggr:   
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'

    #for a in range(min_range, round(max_range)+2):
    #    plt.axvline(x=a, color="red")
        
    plt.savefig(plots_prefix_raster+"P"+str(position)+"_aggr_spontaneous.pdf")
    plt.close()
    
    #ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
'''

In [ ]:
# Attack
'''
for position in positions_rasters_aggr:   
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == attack) & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'

    #for a in range(min_range, round(max_range)+2):
    #    plt.axvline(x=a, color="red")
        
    plt.savefig(plots_prefix_raster+"P"+str(position)+"_aggr_attack.pdf")
    plt.close()
    
    #ax.xaxis.set_major_locator(ticker.MultipleLocator(25))
'''

### 4. Metric 3: shifts of spikes

### 4.1. Metric 3: number of shifts (global - unify all 3 layers)
> One single plot representing the duration of the complete simulation

In [ ]:
plots_prefix_shifts_global = plots_prefix_shifts+"global/"

if not os.path.exists(plots_prefix_shifts_global):
    os.makedirs(plots_prefix_shifts_global)

In [ ]:
df_stats_global = pd.read_csv(stats_neurons_global_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()

In [ ]:
counter = 1

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
label_size = 50
ticks_size = 45
legend_size = 40
line_size = 20

for paramI in list_paramI:
    fig, ax = plt.subplots(figsize=(40,20))

    if attack in ["FLO"]:
        for incr in list_vIncrement:
            ax = sns.lineplot(x=x_plot, y="glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.vIncrement == incr)], linewidth=line_size, label="FLO " + str(incr) + "mV")

        ax.set_xlabel("Number of attacked neurons",fontsize=label_size, fontweight='bold')
        
        # Ranges 
        ax.set_xlim(5,105)
        ax.set_xticks(range(5,110, 5))
        
    elif attack in ["SCA"]:
        ax = sns.lineplot(x=x_plot, y="glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=line_size, label="SCA")

        ax.set_xlabel("Voltage increment (mV)",fontsize=label_size, fontweight='bold')
        
        # Ranges 
        ax.set_xlim(5,60)
        ax.set_xticks(range(5,65,5))

    ax.set_ylabel("Mean percentage of shifts",fontsize=label_size, fontweight='bold')

    #ax.axes.set_title("Global (unification all layers) | " + attack + " | Mean shifts - global perspective",fontsize=50) 

    ax.tick_params(labelsize=ticks_size)
    leg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=label_size)  
    leg.get_frame().set_edgecolor('black')

    fig.savefig(plots_prefix_shifts_global+str(counter)+" Global_mean_shifts_global_perspective.pdf")
    plt.close()

    counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    fig, ax = plt.subplots(1, 1)

    if attack in ["FLO"]:
        for incr in list_vIncrement:
            ax = sns.lineplot(x=x_plot, y="attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

        ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

        cont = 5
        for a in range(0, 22):
            plt.axvline(x=cont*a, color="red")

    elif attack in ["SCA"]:
        ax = sns.lineplot(x=x_plot, y="attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI)], linewidth=10, label="Attack")

        ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

        cont = 5
        for a in range(0, 13):
            plt.axvline(x=cont*a, color="red")

    ax.set_ylabel("Spikes mean",fontsize=30, fontweight='bold')

    ax.axes.set_title("Global (unification all layers) | " + attack + " | Mean shifts - attacked neurons perspective",fontsize=50) 

    ax.tick_params(labelsize=30)
    eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

    ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

    fig.savefig(plots_prefix_shifts_global+str(counter)+" Global_mean_shifts_attacked_neurons_perspective.pdf")
    plt.close()

    counter+=1
'''

### 2.2. Metric 1: mean of shifts (layers)
> 3x3 grid of plots \
> **Rows**: each layer \
> **Columns**: each one of the 3 positions considered

In [ ]:
plots_prefix_shifts_layers = plots_prefix_shifts+"layers/"

if not os.path.exists(plots_prefix_shifts_layers):
    os.makedirs(plots_prefix_shifts_layers)

In [ ]:
df_stats_global = pd.read_csv(stats_neurons_layers_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()

In [ ]:
counter = 1

#### Layer 1

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l1_glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l1_glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Shifts % mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 1 | Position " + str(position) + " | " + attack + " | Mean shifts - global perspective",fontsize=50) 
                          
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_shifts_layers+str(counter)+" L1_P"+str(position)+"_mean_shifts_global_perspective.pdf")
        plt.close()

        counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l1_attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l1_attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Shifts % mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 1 | Position " + str(position) + " | " + attack + " | Mean shifts - attacked neurons perspective",fontsize=50) 
        
        
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_shifts_layers+str(counter)+" L1_P"+str(position)+"_mean_shifts_attacked_perspective.pdf")
        plt.close()

        counter+=1
'''

#### Layer 2

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l2_glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l2_glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Shifts % mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 2 | Position " + str(position) + " | " + attack + " | Mean shifts - global perspective",fontsize=50) 
                          
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_shifts_layers+str(counter)+" L2_P"+str(position)+"_mean_shifts_global_perspective.pdf")
        plt.close()

        counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l2_attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l2_attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Shifts % mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 2 | Position " + str(position) + " | " + attack + " | Mean shifts - attacked neurons perspective",fontsize=50) 
        
        
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_shifts_layers+str(counter)+" L2_P"+str(position)+"_mean_shifts_attacked_perspective.pdf")
        plt.close()

        counter+=1
'''

#### Layer 3

#### > Global perspective (attacked and non-attacked neurons of the layer)

In [ ]:
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l3_glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l3_glbl_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Shifts % mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 3 | Position " + str(position) + " | " + attack + " | Mean shifts - global perspective",fontsize=50) 
                          
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_shifts_layers+str(counter)+" L3_P"+str(position)+"_mean_shifts_global_perspective.pdf")
        plt.close()

        counter+=1

#### > Attacked neurons perspective (only attacked neurons of the layer)

In [ ]:
'''
for paramI in list_paramI:
    for position in range(1,4):
    
        fig, ax = plt.subplots(1, 1)

        if attack in ["FLO"]:
            for incr in list_vIncrement:
                ax = sns.lineplot(x=x_plot, y="l3_attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr))

            ax.set_xlabel("Number of attacked neurons",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 22):
                plt.axvline(x=cont*a, color="red")

        elif attack in ["SCA"]:
            ax = sns.lineplot(x=x_plot, y="l3_attk_shifts_mean", data=df_stats[(df_stats.paramI == paramI) & (df_stats.t_window == position)], linewidth=10, label="Attack")

            ax.set_xlabel("v_increment",fontsize=30, fontweight='bold')

            cont = 5
            for a in range(0, 13):
                plt.axvline(x=cont*a, color="red")

        ax.set_ylabel("Shifts % mean",fontsize=30, fontweight='bold')

        ax.axes.set_title("Layer 3 | Position " + str(position) + " | " + attack + " | Mean shifts - attacked neurons perspective",fontsize=50) 
        
        
        ax.tick_params(labelsize=30)
        eg = ax.legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=30)        

        ax.xaxis.set_major_locator(ticker.MultipleLocator(10))

        fig.savefig(plots_prefix_shifts_layers+str(counter)+" L3_P"+str(position)+"_mean_shifts_attacked_perspective.pdf")
        plt.close()

        counter+=1
'''

### Grid 3x3 Shifts

In [ ]:
plots_prefix_shifts_layers = plots_prefix_shifts+"layers/"

if not os.path.exists(plots_prefix_shifts_layers):
    os.makedirs(plots_prefix_shifts_layers)

df_stats_global = pd.read_csv(stats_neurons_layers_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()

In [ ]:
fig,axs = plt.subplots(nrows=3, ncols=3, sharey='row', sharex='col', figsize=(80,40))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)

position_text = ""

fontsize_label = 40
ticks_size = 30

y_plot_initial = ""
y_plot_attack = ""

for layer in range(1,4):
    axs_row = axs[layer-1]
    
    if layer == 1:
        y_plot_attack = "l1_glbl_shifts_mean"
    elif layer == 2:
        y_plot_attack = "l2_glbl_shifts_mean"
    elif layer == 3:
        y_plot_attack = "l3_glbl_shifts_mean"
      
    for position in range(1, 4):
        if position == 1:
            position_text = "Position 1"
        elif position == 2:
            position_text = "Position 13"
        elif position == 3:
            position_text = "Position 27"
                
        if attack in ["FLO"]:
            for incr in list_vIncrement:
                sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=10, label="Attack- Increment: " + str(incr), ax=axs_row[position-1])

            cont = 5
            for a in range(0, 22):
                axs_row[position-1].axvline(x=cont*a, color="red")
            
            axs_row[position-1].set_xlabel(position_text+" - Number of attacked neurons", fontsize=fontsize_label, fontweight="bold")

        elif attack in ["SCA"]:
            sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position)], linewidth=10, label="Attack", ax=axs_row[position-1])

            cont = 5
            for a in range(0, 13):
                axs_row[position-1].axvline(x=cont*a, color="red")
                
            axs_row[position-1].set_xlabel(position_text+" - Voltage increment", fontsize=fontsize_label, fontweight="bold")
  
        if layer!=1 or position !=1 :
            axs_row[position-1].get_legend().remove()
        
        axs_row[position-1].set_ylabel("Layer "+str(layer)+" - Mean shifts %", fontsize=fontsize_label, fontweight="bold")
        axs_row[position-1].tick_params(labelsize=ticks_size)
        
fig.savefig(plots_prefix_shifts_layers+"grid_3_3_shifts.pdf")
plt.close()

### Grid 3x1 Shifts

In [ ]:
plots_prefix_shifts_layers = plots_prefix_shifts+"layers/"

if not os.path.exists(plots_prefix_shifts_layers):
    os.makedirs(plots_prefix_shifts_layers)

df_stats_global = pd.read_csv(stats_neurons_layers_filename, delimiter=";")
#df_stats_global.fillna(0)

df_stats = df_stats_global[df_stats_global.n_attacks == 1]

list_vIncrement = df_stats.vIncrement.unique()
list_paramI = df_stats.paramI.unique()

In [ ]:
label_size = 50
ticks_size = 45
legend_size = 40
line_size = 20

fig,axs = plt.subplots(nrows=3, ncols=1, sharey='row', sharex='col', figsize=(40, 30))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.04)

position_text = ""

y_plot_initial = ""
y_plot_attack = ""

for layer in range(1,4):
    
    if layer == 1:
        y_plot_attack = "l1_glbl_shifts_mean"
    elif layer == 2:
        y_plot_attack = "l2_glbl_shifts_mean"
    elif layer == 3:
        y_plot_attack = "l3_glbl_shifts_mean"
      
    for position in [3]: #range(1, 4):
        if position == 1:
            position_text = "Position 1"
        elif position == 2:
            position_text = "Position 13"
        elif position == 3:
            position_text = "Position 27"
                
        if attack in ["FLO"]:
            for incr in list_vIncrement:
                sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position) & (df_stats.vIncrement == incr)], linewidth=line_size, label="FLO " + str(incr) + "mV", ax=axs[layer-1])

            # Ranges 
            axs[layer-1].set_xlim(5,105)
            axs[layer-1].set_xticks(range(5,110, 5))
            
            axs[layer-1].set_xlabel("Number of attacked neurons", fontsize=label_size, fontweight="bold")

        elif attack in ["SCA"]:
            sns.lineplot(x=x_plot, y=y_plot_attack, data=df_stats[(df_stats.t_window == position)], linewidth=line_size, label="SCA", ax=axs[layer-1])

            # Ranges 
            axs[layer-1].set_xlim(5,60)
            axs[layer-1].set_xticks(range(5,65, 5))
            
            axs[layer-1].set_xlabel("Voltage increment (mV)", fontsize=label_size, fontweight="bold")

        axs[layer-1].set_ylabel("Mean % shifts (L"+str(layer)+")", fontsize=label_size, fontweight="bold")
        axs[layer-1].tick_params(labelsize=ticks_size)
        
        axs[layer-1].legend(loc='best', fancybox=True, shadow=True, ncol=1, frameon='True',fontsize=legend_size)
        axs[layer-1].get_legend().get_frame().set_edgecolor('black')
        
        if layer!=1:
            axs[layer-1].get_legend().remove()
        
fig.savefig(plots_prefix_shifts_layers+"grid_1_3_shifts.pdf")
plt.close()

#### Generate test data

In [ ]:
# Generate synapsis12
'''
for i in range(0,200):
    for j in range(0,72):
        print(str(i)+";"+str(j)+";"+str(round(random.uniform(0.01, 0.05), 2)))
'''

In [ ]:
# Generate synapsis23
'''
for i in range(0,72):
    for j in range(0,4):
        print(str(i)+";"+str(j)+";"+str(round(random.uniform(0.01, 0.05), 2)))
'''

In [ ]:
# Currently, between 10 and 15 (*mV/ms)
'''
def generateStoreIzhikevichParamI(fileName, lowerValue, higherValue):
    with open(fileName, 'w') as writer:
        for i in range(0, 276):
            writer.write(str(round(random.uniform(lowerValue, higherValue), 2))+'\n')  
    
generateStoreIzhikevichParamI("paramI.txt", 10, 15)
'''

In [ ]:
# Generate values for initial voltage
'''
def generate_values_initial_voltage(fileName, lowerValue, higherValue):
    with open(fileName, 'w') as writer:
        for i in range(0, 276):
            writer.write(str(round(random.uniform(lowerValue, higherValue), 2))+'\n')

generate_values_initial_voltage("initial_voltage.txt", -65, 0)
'''

#### VISUAL analysis

In [ ]:
# Visualize the aggr of spikes of first position: SPONTANEOUS and FLO

for position in [0]:   
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("Spontaneous - Position " + str(position))
    
  
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("FLO - Position " + str(position))    

    
    
    # Subplots with common y axis
    fig,axs = plt.subplots(nrows=1, ncols=2, sharey='row', figsize=(80,40))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[0]) #, style='dataset'
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[1]) #, style='dataset'

In [ ]:
# Visualize the aggr of spikes of 12th position: SPONTANEOUS and FLO

for position in [12]:   
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("Spontaneous - Position " + str(position))
    

    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("FLO - Position " + str(position))    


    
    fig,axs = plt.subplots(nrows=1, ncols=2, sharey='row', figsize=(80,40))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[0]) #, style='dataset'
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[1]) #, style='dataset'

In [ ]:
# Visualize the aggr of spikes of 13th position: SPONTANEOUS and FLO

for position in [13]:   
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("Spontaneous - Position " + str(position))
    
    
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("FLO - Position " + str(position))    

    
    
    fig,axs = plt.subplots(nrows=1, ncols=2, sharey='row', figsize=(80,40))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[0]) #, style='dataset'
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[1]) #, style='dataset'

In [ ]:
# Visualize the aggr of spikes of last position: SPONTANEOUS and FLO

for position in [26]:   
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("Spontaneous - Position " + str(position))
    

    
    
    fig,ax = plt.subplots()
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80) #, style='dataset'
    
    ax.set_title("FLO - Position " + str(position))    
    
    
    
    fig,axs = plt.subplots(nrows=1, ncols=2, sharey='row', figsize=(80,40))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=0.03)
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "initial_state") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[0]) #, style='dataset'
    
    sns.scatterplot(x='time_delta', y='number_spikes', data=df[(df.attack == "FLO") & (df.position == position)],
                    hue='attack', palette=["#092C48"], s=80, ax=axs[1]) #, style='dataset'

#### Total number of spikes

In [ ]:
sum_spikes_initial = []
sum_spikes_attack = []

for position in list_positions:
    sum_spikes_initial.append(df[(df.position == position) & (df.attack == "initial_state")]["number_spikes"].sum())
    sum_spikes_attack.append(df[(df.position == position) & (df.attack == "FLO")]["number_spikes"].sum())

In [ ]:
# Number of spikes per position
fig, ax = plt.subplots(1,1)

sns.lineplot(data=np.array(sum_spikes_initial), linewidth=10, label="Spontaneous", ax=ax)
sns.lineplot(data=np.array(sum_spikes_attack), linewidth=10, label="FLO", ax=ax)   

cont = 5
for a in list_positions:
    ax.axvline(x=a, color="red")

ax.set_xlabel("Positions")
ax.set_ylabel("Total number of spikes")

#### Vertical analysis

In [ ]:
n_instants_with_spikes_initial = []
n_instants_with_spikes_attack = []

n_instants_with_spikes_percent_initial = []
n_instants_with_spikes_percent_attack = []

for position in list_positions:
    n_instants_with_spikes_initial.append(df[(df.position == position) & (df.attack == "initial_state")]["time_delta"].count())
    n_instants_with_spikes_attack.append(df[(df.position == position) & (df.attack == "FLO")]["time_delta"].count())
    
    n_instants_with_spikes_percent_initial.append(df[(df.position == position) & (df.attack == "initial_state")]["time_delta"].count()/10000*100)
    n_instants_with_spikes_percent_attack.append(df[(df.position == position) & (df.attack == "FLO")]["time_delta"].count()/10000*100)

In [ ]:
# NUMBER of instants with spikes, per position
fig, ax = plt.subplots(1,1)
sns.lineplot(data=np.array(n_instants_with_spikes_initial), linewidth=10, label="Spontaneous", ax=ax)
sns.lineplot(data=np.array(n_instants_with_spikes_attack), linewidth=10, label="FLO", ax=ax)

cont = 5
for a in list_positions:
    ax.axvline(x=a, color="red")

ax.set_xlabel("Positions")
ax.set_ylabel("Number of instants with spikes")

# PERCENTAGE of instants with spikes, per position
fig, ax = plt.subplots(1,1)
sns.lineplot(data=np.array(n_instants_with_spikes_percent_initial), linewidth=10, label="Spontaneous", ax=ax)
sns.lineplot(data=np.array(n_instants_with_spikes_percent_attack), linewidth=10, label="FLO", ax=ax)

cont = 5
for a in list_positions:
    ax.axvline(x=a, color="red")

ax.set_xlabel("Positions")
ax.set_ylabel("Percentage of instants with spikes")

#### Horizontal analysis

In [ ]:
def patch_violinplot():
    from matplotlib.collections import PolyCollection
    ax = plt.gca()
    counter = 0
    for art in ax.get_children():
        if isinstance(art, PolyCollection):
            if counter % 2 == 0:
                art.set_edgecolor("#4C72B0")
            else:
                art.set_edgecolor("#DD8452")
        
            counter+=1

ax = sns.violinplot(x="position", y="number_spikes", hue="attack", cut=0, data=df) # split=True
patch_violinplot()

plt.savefig("vertical_dispersion.pdf")
plt.close()